In [ ]:
sm = snakemake

In [ ]:
import spherpro.bro as spb
import spherpro.datastore as spd
import spherpro.library as spl
import spherpro.configuration as conf
import spherpro.db as db
import sqlalchemy as sa
import imp
import pycytools as pct
import pycytools.library
import re
import os
import pandas as pd
import numpy as np
import plotnine as gg
import spherpro.library as lib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import pdb
import pathlib
import colorcet

%matplotlib notebook
%matplotlib inline


import seaborn as sns
import scipy.stats as stats
from IPython.display import display

from pandas.api.types import CategoricalDtype

In [ ]:
from src.variables import Vars
from src.config import Conf

# Aim:

Use the results from the variability analysis show that variability should not be seen as an additive effect.




## Variability analysis
Changes: try log10(x+0.1)


In [ ]:
import spherpro.bromodules.helpers_vz as helpers_vz
imp.reload(helpers_vz)

In [ ]:
def get_valid_filename(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

In [ ]:
class CurVariableHelper(Vars):
    COL_D2RIM = 'DistRim'
    COL_GENE = 'gene'
    COL_GENE_UNTAGGED = 'gene_untagged'
    COL_TAG = 'tag'
    COL_DOXO = 'doxocycline'
    COL_DILUTION = 'dilution'
    COL_GOODNAME = 'goodname'
    COL_ISNB = 'isnb'
    COL_FITTED = 'fitted'
    COL_RESID = 'residual'
    COL_IMGID = db.images.image_id.key
    COL_OBJ_NR = db.objects.object_number.key
    COL_FC = 'fc'
    COL_P = 'p'
    COL_DF = 'DF'
    COL_DELTA = 'delta'
    COL_TSTAT = 't'
    COL_TAGSTAT = 'TagStat'
    COL_POSSTAT = 'PosStat'
    COL_WORKING = 'working'
    COL_N = 'n'
    COL_N_OVEREXPR = 'n_overexpr'
    COL_FC_CENS = 'fc_cens'
    COL_NB = 'nb'
    COL_P_CORR = 'p_corrected'
    COL_ISSIG = 'is_sig_sel'
    COL_FITCONDITIONNAME = 'FitConditionName'
    COL_VALUES = db.object_measurements.value.key

    COL_COEFNAME = 'coefname'
    
    LAB_CELLLINE = 'Cellline'
    LAB_TP = 'Timepoint'
    LAB_CONC = 'Size'
    
V = CurVariableHelper

In [ ]:
V.COL_MODTYPE = 'modtype'
V.COL_MODEL = 'model'
V.COL_MODELCLASS = 'modelclass'
V.COL_METAL = 'metal'
V.COL_MARKER_CLASS = 'marker_class'
V.COL_IS_CC = 'is_cc'

V.COL_CONDITION = V.COL_CONDNAME
V.COL_TP = 'timepoint'
V.COL_CONC = 'concentration'
V.COL_CELLLINE = 'cellline'

V.COL_CHANNEL = 'channel'

In [ ]:
class C(Conf):  
    main_conds = sm.params.main_conds
    main_conds_cl = sm.params.celllines
    all_conds =  sm.params.all_conds
    fig2_cond = [main_conds[1]]
    fn_config = sm.input.fn_config

In [ ]:
V.COL_MEASTYPE = db.measurement_types.measurement_type.key
V.COL_MEASID = db.measurements.measurement_id.key
V.COL_VALUE = db.object_measurements.value.key
MEAS_INTENSITY = 'Intensity'

In [ ]:
cmap_cells = [colors.to_hex(c) for c in sns.color_palette('deep')]

In [ ]:
cmap_cells

In [ ]:
# I added consoring as there were hugh outliers in the data
def censor_dat(x, q=99.9):
    x = np.copy(x)
    p = np.percentile(x,q=q)
    x[x> p] = p
    return x

def cur_logtransf(x):
    return np.log10(x+0.1)

def cur_transf(x):
    x= censor_dat(x)
    x= cur_logtransf(x)
    return x

def transf_intensities(dat, dat_measmeta):
    ids = dat_measmeta.loc[dat_measmeta[V.COL_MEASTYPE] == MEAS_INTENSITY, V.COL_MEASID]
    fil = dat[V.COL_MEASID].isin(ids)
    dat = dat.copy()
    dat.loc[fil, V.COL_VALUE] = dat.loc[fil, :].groupby(V.COL_MEASID).transform(cur_transf)
    return dat

def get_imgs_for_cond(condition_name):
    q = (bro.session.query(db.images.image_id)
     .join(db.conditions)
     .join(db.valid_images)
     .filter(db.conditions.condition_name.like(f'{condition_name}%')))
    return [i[0] for i in q.all()] 

def get_condids_for_cond(condition_name):
    q = (bro.session.query(db.conditions.condition_id)
     .join(db.images)
     .join(db.valid_images)
     .filter(db.conditions.condition_name.like(f'{condition_name}%')))
    return [i[0] for i in q.all()] 

In [ ]:
fn_config = C.fn_config
fn_modstats_out = sm.input.fn_modstats
fn_modstats_perrep = sm.input.fn_modstats_perrep
fol_out_paper = pathlib.Path(sm.output.fol_out)
fn_params = sm.input.fn_modstats_params
fol_out = pathlib.Path(sm.output.fol_out)
os.makedirs(fol_out, exist_ok=True)

In [ ]:
dat_pannel_ord = pd.read_csv(sm.input.fn_panel_ord)

In [ ]:




bro = spb.get_bro(fn_config)


hpr = helpers_vz.HelperVZ(bro)

dat_pannelcsv = hpr.get_pannelcsv()


dat_pannelcsv.loc[dat_pannelcsv['metal'] == 'Gd155',V.COL_WORKING] = True

dat_measmeta = hpr.get_measuremeta(dat_pannelcsv,additional_measfilt=sa.and_(db.stacks.stack_name == 'Dist',
                                                                            db.measurements.measurement_name == 'dist-rim',
                                                                            db.ref_planes.channel_name == 'object'))

dat_imgmeta = hpr.get_imgmeta()
#dat_imgmeta[V.COL_CONDLEVEL] = dat_imgmeta[V.COL_CONDID].map(str)
#dat_imgmeta[V.COL_SITELEVEL] = dat_imgmeta[V.COL_SITEID].map(str)


fil_good_meas = hpr.get_fil_good_meas(dat_measmeta)



#dat_measmeta = dat_measmeta.merge(dat_pannelcsv[[V.COL_METAL, V.COL_MARKER_CLASS]],how='left')
dat_measmeta = dat_measmeta.merge(dat_pannelcsv[[V.COL_METAL, V.COL_IS_CC]], how='left')
dat_measmeta[V.COL_IS_CC] = dat_measmeta[V.COL_IS_CC] == 1
#dat_measmeta[V.COL_IS_CC] = False
dat_measmeta = dat_measmeta.set_index(V.COL_MEASID, drop=False)

In [ ]:
fol_out_paper.mkdir(exist_ok=True)

In [ ]:
dat_layout = pd.read_csv(bro.data.conf['layout_csv']['path'])

In [ ]:
dat_mod_stats = pd.read_csv(fn_modstats_out)

In [ ]:
r_var= V.COL_R2 = 'r2_adj_corr'

In [ ]:
import scipy.cluster.hierarchy as hclust
import scipy.spatial.distance as dist
import scipy.stats as spstats

In [ ]:
cluster_models = ['dist', 'nb', 'self', 'cc','int']

## Make names for plotting

In [ ]:

COL_CLASS = 'class'
COL_ORD_WITHIN = 'ord_within_class'
cols = [V.COL_METAL, COL_CLASS, COL_ORD_WITHIN]
class_ord = ['tag','egf','mtor', 'cellcycle', 'stress', 'apoptosis','total']
dat_pannel_ord[COL_CLASS] = pd.Categorical(dat_pannel_ord[COL_CLASS], categories=class_ord)

In [ ]:
name_d = {c: n for c, n in zip(dat_measmeta[V.COL_CHANNELNAME], dat_measmeta[V.COL_GOODNAME])}

In [ ]:
dat_mod_stats[V.COL_GOODNAME] = dat_mod_stats[V.COL_CHANNELNAME].replace(name_d)

In [ ]:
V.COL_IDCHANNAME = 'channel'

In [ ]:
dat_mod_stats[V.COL_IDCHANNAME] = dat_mod_stats.apply(lambda x: x[V.COL_GOODNAME], axis=1)

In [ ]:
ord_pltname = dat_mod_stats.merge(dat_pannel_ord[[V.COL_METAL, COL_CLASS, COL_ORD_WITHIN]], left_on=V.COL_CHANNELNAME,
                             right_on=V.COL_METAL).sort_values([ COL_CLASS, COL_ORD_WITHIN])[V.COL_IDCHANNAME].unique()

In [ ]:
dat_mod_stats[V.COL_IDCHANNAME] = pd.Categorical(dat_mod_stats[V.COL_IDCHANNAME], categories=ord_pltname )

In [ ]:
ord_pltname

## Determine clustering of markers via clustering of the R2 values

In [ ]:
def run_r2_clustering(dat):
    cdat = (dat
        .pivot_table(columns=V.COL_IDCHANNAME, values=V.COL_R2,
                     index=[V.COL_MODELCLASS, V.COL_CONDNAME],
                     aggfunc=np.median))

    rlink =  hclust.linkage(cdat.T, metric='correlation', method='complete',optimal_ordering=True)
    labels = cdat.columns
    return rlink, labels

def run_r2_clustering_cond(dat):
    cdat = (dat
        .pivot_table(columns=[V.COL_MODELCLASS,V.COL_IDCHANNAME], values=V.COL_R2, index=[ V.COL_CONDNAME],
                                                                                            aggfunc=np.mean))

    rlink =  hclust.linkage(cdat, metric='correlation', method='average',optimal_ordering=True)
    labels = cdat.index
    return rlink, labels

def get_order_from_clust(rlink, labels):
    return labels[hclust.leaves_list(rlink)]

In [ ]:
cluster_models

In [ ]:
rlink, labs = run_r2_clustering(dat_mod_stats.loc[dat_mod_stats[V.COL_MODELCLASS].isin(cluster_models), :])

#marklvl = (get_order_from_clust(rlink, labs))

fig = plt.figure(figsize=(15,0.5))
clust_res = hclust.dendrogram(rlink,labels=labs,
                              leaf_rotation=90,color_threshold=0.4)

In [ ]:
fn_modstats_perrep

In [ ]:
fig.savefig(fol_out_paper/'marker_dendrogram.pdf')

In [ ]:
rlink, labs = run_r2_clustering_cond(dat_mod_stats.loc[dat_mod_stats[V.COL_MODELCLASS].isin(cluster_models), :])

condlevel = get_order_from_clust(rlink, labs)
plt.figure(figsize=(10,20))
l = hclust.dendrogram(rlink,labels=labs, leaf_rotation=0, orientation="left",color_threshold=3)

In [ ]:
def to_cat(x):
    return pd.Categorical(x, categories=sorted(np.unique(x)))

In [ ]:
dat_layout.columns


dat_layout=dat_layout.rename(columns={bro.data.conf['layout_csv']['condition_col']: V.COL_CONDNAME})

dat_condmeta=(dat_layout.groupby(by=[V.COL_CELLLINE, V.COL_CONC, V.COL_TP])['well'].count()
   .reset_index()
   .assign(**{V.COL_CONDNAME: 
              lambda x: x.apply(
                  lambda g: f'{g[V.COL_CELLLINE]}_c{g[V.COL_CONC]}_tp{g[V.COL_TP]}', axis=1)}))

dat_condmeta[V.COL_CELLLINE] = pd.Categorical(dat_condmeta[V.COL_CELLLINE],
                                              categories=C.main_conds_cl)

In [ ]:
class M:
    "contains the strings of the different models"
    dist = 'dist'
    nb = 'nb'
    int = 'int'
    cc ='cc'
    self = 'self'
    dist_nb = '_'.join([dist, nb])
    dist_nb_self = '_'.join([dist, nb, self])
    nb_self = '_'.join([nb, self])
    dist_int_nb_self = '_'.join([dist,int, nb, self])
    
dat_mod_stats_wide = (dat_mod_stats.pivot_table(columns = V.COL_MODELCLASS, index=[V.COL_IDCHANNAME, V.COL_CONDITION], values=V.COL_R2)
 .reset_index() 

) 

In [ ]:
V.COL_MODELCLASS_F = V.COL_MODELCLASS+'_factor'
dat_mod_stats[V.COL_MODELCLASS_F] =  dat_mod_stats[V.COL_MODELCLASS].astype(CategoricalDtype(
                                                                        categories=
                                                                        ["dist_int_nb_self",
                                                                         'int',
                                                                         'cc',
                                                                         "dist_nb_self",
                                                                         "dist_nb",
                                                                         'nb_self',
                                                                         'self',
                                                                         'nb',
                                                                         'nbcc',
                                                                         'dist'
                                                                         ]))

In [ ]:
fil = dat_mod_stats[V.COL_MODELCLASS].isin(['dist_int_nb_self'])
(gg.ggplot(dat_mod_stats.loc[fil], gg.aes(x=V.COL_IDCHANNAME, y=V.COL_R2))+
           #gg.facet_grid('.~'+V.COL_IDCHANNAME, scales='free_x')+
          # gg.facet_grid(+'~.')+
              gg.geom_hline(gg.aes(yintercept=1))+
             gg.geom_hline(gg.aes(yintercept=0))+
              gg.geom_line(gg.aes(group=V.COL_IDCHANNAME), alpha=0.5)+
 gg.geom_point(gg.aes(color=V.COL_CONDNAME), position=gg.position_dodge(width=0.4))+
           gg.expand_limits(y=1)+
            gg.ggtitle('Prediction R2 of full model')+
             gg.ylab('R2 [explained variance]')+
             gg.xlab('')+
           gg.theme(axis_text_x = gg.element_text(angle = 90, hjust = 0.1), figure_size=(6,3)
            )
)

In [ ]:
meas_ids = list((dat_mod_stats.merge(dat_measmeta
                    .query(f'{db.measurements.measurement_name.key} == "MeanIntensityComp"'))
            [db.measurements.measurement_id.key]
           ).unique())

In [ ]:
from sqlalchemy import func

In [ ]:
def get_q(c):
    q = (bro.session.query(db.object_measurements.measurement_id,
                          func.percentile_cont(0.5).within_group(
                        db.object_measurements.value.asc()).label(db.object_measurements.value.key))
         .join(db.measurements,
               db.object_measurements.measurement_id==db.measurements.measurement_id)
         .join(db.planes)
          .join(db.ref_stacks, db.ref_stacks.ref_stack_id == db.planes.ref_stack_id)
         .join(db.objects, db.object_measurements.object_id== db.objects.object_id)
         .join(db.images)
         .join(db.conditions)
         .join(db.valid_objects)
         .join(db.valid_images)
         .filter(db.conditions.condition_name.like(c))
         .filter(db.object_measurements.measurement_id.in_([int(i) for i in meas_ids]))
         .group_by(db.object_measurements.measurement_id, db.ref_stacks.scale)
        )
    return q



In [ ]:
meas_ids

In [ ]:
%%time
q_obj = (bro.data.get_objectmeta_query()
         .filter(db.objects.object_type == 'cell')
         .join(db.conditions, db.conditions.condition_id==db.images.condition_id)
         .add_columns(db.conditions.condition_name)
        )

q_meas = (bro.data.get_measmeta_query()
         .filter(db.measurements.measurement_id.in_([int(m) for m in meas_ids]))
         .add_column(db.ref_stacks.scale)
         .add_column(db.ref_planes.channel_name))

adat_objmeas = bro.io.objmeasurements.get_measurements(q_obj=q_obj, q_meas=q_meas)
adat_objmeas = bro.io.objmeasurements.scale_anndata(adat_objmeas)

In [ ]:
def get_conds(conpat):
    return [r[0] for r in bro.session.query(db.conditions.condition_name)
         .filter(db.conditions.condition_name.like(conpat))]

In [ ]:
def get_median(adat, c):
    return np.median(adat.X[adat.obs[V.COL_CONDNAME].isin(get_conds(c)),:], axis=0)

c_names = dat_mod_stats[V.COL_CONDNAME].unique()
dat_med =(pd.DataFrame([get_median(adat_objmeas, c) for c in c_names], index=pd.Index(c_names, name=V.COL_CONDNAME), columns=adat_objmeas.var[V.COL_CHANNELNAME]
                  )
          .stack()
          .rename(V.COL_VALUE)
          .reset_index())
dat_med

In [ ]:
dat_mod_stats[V.COL_MODELCLASS]

In [ ]:
dat_mod_stats.loc[fil]

In [ ]:
dat_condmeta

def is_total(x):
    return x.map(lambda x: (x.startswith('t-')| x.startswith('DNA')))

In [ ]:
fil = dat_mod_stats[V.COL_MODELCLASS].isin(['dist_int_nb_self'])
p=(gg.ggplot(dat_mod_stats.loc[fil]
           .merge(dat_condmeta)
           .merge(dat_med), gg.aes(x=f'np.log10({V.COL_VALUE}+0.1)', y=V.COL_R2))+
           #gg.facet_grid('.~'+V.COL_IDCHANNAME, scales='free_x')+
          # gg.facet_grid(+'~.')+
              gg.geom_hline(gg.aes(yintercept=1))+
             gg.geom_hline(gg.aes(yintercept=0))+
 gg.geom_point(gg.aes(color=V.COL_CELLLINE, size=f'{V.COL_CONC}')
              #        shape=f'to_cat({V.COL_TP})')
              )+
             gg.geom_smooth(gg.aes(group=1),method='loess')+
           gg.expand_limits(y=1)+
 gg.scale_size(range=(1,2), breaks=sorted(dat_condmeta[V.COL_CONC].unique()))+
            gg.ggtitle('Prediction R2 of full model')+
             gg.ylab('R2 [explained variance]')+
             gg.xlab('Median channel intensity [log10(counts per pixel)]')+
             gg.scale_color_manual(cmap_cells)+
           gg.theme(axis_text_x = gg.element_text(angle = 90, hjust = 0.1), figure_size=(6,3)
            )
)
p

In [ ]:
cmap_cells

In [ ]:
fil = dat_mod_stats[V.COL_MODELCLASS].isin(['dist_int_nb_self'])
p=(gg.ggplot(dat_mod_stats.loc[fil]
           .merge(dat_condmeta)
           .merge(dat_med), gg.aes(x=f'np.log10({V.COL_VALUE}+0.1)', y=V.COL_R2))+
           #gg.facet_grid('.~'+V.COL_IDCHANNAME, scales='free_x')+
          # gg.facet_grid(+'~.')+
              gg.geom_hline(gg.aes(yintercept=1))+
             gg.geom_hline(gg.aes(yintercept=0))+
 gg.geom_point(gg.aes(color=V.COL_CELLLINE, size=f'{V.COL_CONC}',
                      shape=f'to_cat({V.COL_TP})'))+
             gg.geom_smooth(gg.aes(group=1),method='loess')+
           gg.scale_y_continuous(limits=(0,1),expand=(0,0.01))+
 gg.scale_size(range=(1,2), breaks=sorted(dat_condmeta[V.COL_CONC].unique()))+
            gg.ggtitle('Prediction R2 of full model')+
             gg.ylab('R2 [explained variance]')+
                         gg.xlab('Median channel intensity [log10(counts per pixel)]')+
             gg.scale_color_manual(cmap_cells)+
           gg.theme(axis_text_x = gg.element_text(angle = 90, hjust = 0.1), figure_size=(6,3)
            )
)
gg.ggsave(p, fol_out_paper/'subfig_r2_vs_median.pdf')
p

In [ ]:
fil = dat_mod_stats[V.COL_MODELCLASS].isin(['dist_int_nb_self'])
p=(gg.ggplot(dat_mod_stats.loc[fil]
           .merge(dat_condmeta)
           .merge(dat_med)
              .assign(**{'grp': 
              lambda x: x.apply(
                  lambda g: f'{g[V.COL_CELLLINE]}c{g[V.COL_CHANNELNAME]}', axis=1)}),
              gg.aes(x=f'np.log10({V.COL_VALUE}+0.1)', y=V.COL_R2))+
        # gg.facet_wrap(f'{V.COL_IDCHANNAME}',ncol=1)+
          # gg.facet_grid(+'~.')+
              gg.geom_hline(gg.aes(yintercept=1))+
             gg.geom_hline(gg.aes(yintercept=0))+
     gg.geom_line(gg.aes(color=V.COL_CELLLINE, group='grp'), alpha=0.3)+
 gg.geom_point(gg.aes(color=V.COL_CELLLINE, size=f'{V.COL_CONC}',
                      shape=f'to_cat({V.COL_TP})'))+
             #gg.geom_smooth(gg.aes(group=1),method='loess')+
           gg.expand_limits(y=1)+
 gg.scale_size(range=(1,2), breaks=sorted(dat_condmeta[V.COL_CONC].unique()))+
            gg.ggtitle('Prediction R2 of full model')+
             gg.ylab('R2 [explained variance]')+
             gg.xlab('')+
             gg.scale_color_manual(cmap_cells)+
           gg.theme(axis_text_x = gg.element_text(angle = 90, hjust = 0.1),
                    figure_size=(6,3),
                    strip_text_y = gg.element_text(angle = 0)
            )
)
p

In [ ]:
(dat_mod_stats.loc[fil][V.COL_R2] > 0.5).describe()

In [ ]:
465/792

In [ ]:
fil = dat_mod_stats[V.COL_MODELCLASS].isin(['dist_int_nb_self'])
(gg.ggplot(dat_mod_stats.loc[fil]
           .merge(dat_condmeta)
           .merge(dat_med)
              .assign(**{'grp': 
              lambda x: x.apply(
                  lambda g: f'{g[V.COL_CELLLINE]}c{g[V.COL_CHANNELNAME]}', axis=1)}),
              gg.aes(x=f'np.log10({V.COL_VALUE}*2**16+0.1)', y=V.COL_R2))+
         gg.facet_wrap(f'{V.COL_IDCHANNAME}',ncol=1)+
          # gg.facet_grid(+'~.')+
              gg.geom_hline(gg.aes(yintercept=1))+
             gg.geom_hline(gg.aes(yintercept=0))+
     gg.geom_line(gg.aes(color=V.COL_CELLLINE, group='grp'))+
 gg.geom_point(gg.aes(color=V.COL_CELLLINE, size=f'{V.COL_CONC}',
                      shape=f'to_cat({V.COL_TP})'))+
             #gg.geom_smooth(gg.aes(group=1),method='loess')+
           gg.expand_limits(y=1)+
 gg.scale_size(range=(1,2), breaks=sorted(dat_condmeta[V.COL_CONC].unique()))+
            gg.ggtitle('Prediction R2 of full model')+
             gg.ylab('R2 [explained variance]')+
             gg.xlab('')+
           gg.theme(axis_text_x = gg.element_text(angle = 90, hjust = 0.1),
                    figure_size=(3,38),
                    strip_text_y = gg.element_text(angle = 0)
            )
)

In [ ]:
fil = dat_mod_stats[V.COL_MODELCLASS].isin(['dist_int_nb_self'])
p = (gg.ggplot(dat_mod_stats.loc[fil].merge(dat_condmeta), gg.aes(x=V.COL_IDCHANNAME, y=V.COL_R2))+
           #gg.facet_grid('.~'+V.COL_IDCHANNAME, scales='free_x')+
          # gg.facet_grid(+'~.')+
              gg.geom_hline(gg.aes(yintercept=1))+
             gg.geom_hline(gg.aes(yintercept=0))+
              #gg.geom_line(gg.aes(group=V.COL_IDCHANNAME, color=V.COL_CELLLINE), alpha=0.5)+
 gg.geom_point(gg.aes(color=V.COL_CELLLINE, size=f'{V.COL_CONC}',
                      shape=f'to_cat({V.COL_TP})'), position=gg.position_dodge(width=0.4))+
           gg.scale_y_continuous(limits=(0,1),expand=(0,0.01))+
 gg.scale_size(range=(1,2), breaks=sorted(dat_condmeta[V.COL_CONC].unique()), name=V.LAB_CONC)+
            gg.ggtitle('Prediction R2 of full model')+
             gg.ylab('R2 [explained variance]')+
             gg.xlab('')+
             gg.scale_color_manual(cmap_cells,name = V.LAB_CELLLINE)+
             gg.scale_shape(name=V.LAB_TP)+
           gg.theme(axis_text_x = gg.element_text(angle = -90, hjust = 0.1), figure_size=(4,2),
                    text=gg.element_text(size=6)
            )
)

fig=p.draw()

In [ ]:
fig.savefig( fol_out_paper/'subfig_r2_full_all.pdf')

fig.savefig fol_out_paper/'subfig_r2_full_all.pdf')

In [ ]:
fil = dat_mod_stats[V.COL_MODELCLASS].isin(['dist','nb', 'dist_nb'])
(gg.ggplot(dat_mod_stats.loc[fil].query(f'{V.COL_CONDNAME} in {C.main_conds}'),
           
           gg.aes(x=V.COL_IDCHANNAME, y=V.COL_R2))+
           gg.facet_grid(V.COL_CONDITION+'~.')+
              gg.geom_hline(gg.aes(yintercept=1))+
             gg.geom_hline(gg.aes(yintercept=0))+ 
           gg.geom_line(gg.aes(group=V.COL_IDCHANNAME), alpha=0.5)+
           gg.geom_point(gg.aes(color=V.COL_MODELCLASS_F), position=gg.position_dodge(width=0.9))+
           gg.expand_limits(y=1)+
            gg.ggtitle('Dist-to-rim vs neighbourhood')+
           gg.theme(axis_text_x = gg.element_text(angle = -90, hjust = 0.1), figure_size=(5,6)
            )
)

In [ ]:
fil = dat_mod_stats[V.COL_MODELCLASS].isin(['dist','nb', 'dist_nb'])
(gg.ggplot(dat_mod_stats.loc[fil].query(f'{V.COL_CONDNAME} in {C.fig2_cond}'),
           
           gg.aes(x=V.COL_IDCHANNAME, y=V.COL_R2))+
           gg.facet_grid(V.COL_CONDITION+'~.')+
              gg.geom_hline(gg.aes(yintercept=1))+
             gg.geom_hline(gg.aes(yintercept=0))+ 
           gg.geom_line(gg.aes(group=V.COL_IDCHANNAME), alpha=0.5)+
           gg.geom_point(gg.aes(color=V.COL_MODELCLASS_F), position=gg.position_dodge(width=0.9))+
           gg.expand_limits(y=1)+
            gg.ggtitle('Dist-to-rim vs neighbourhood')+
           gg.theme(axis_text_x = gg.element_text(angle = -90, hjust = 0.1), figure_size=(5,3)
            )
)

In [ ]:
fil = dat_mod_stats[V.COL_MODELCLASS].isin(['dist_nb','self', 'dist_nb_self'])
(gg.ggplot(dat_mod_stats.loc[fil].query(f'{V.COL_CONDNAME} in {C.fig2_cond}'),
           
           gg.aes(x=V.COL_IDCHANNAME, y=V.COL_R2))+
           gg.facet_grid(V.COL_CONDITION+'~.')+
              gg.geom_hline(gg.aes(yintercept=1))+
             gg.geom_hline(gg.aes(yintercept=0))+ 
           gg.geom_line(gg.aes(group=V.COL_IDCHANNAME), alpha=0.5)+
           gg.geom_point(gg.aes(color=V.COL_MODELCLASS_F), position=gg.position_dodge(width=0.9))+
           gg.expand_limits(y=1)+
            gg.ggtitle('Dist-to-rim vs neighbourhood')+
           gg.theme(axis_text_x = gg.element_text(angle = 90, hjust = 0.1), figure_size=(5,3)
            )
)

In [ ]:
fil = dat_mod_stats[V.COL_MODELCLASS].isin(['dist_nb_self','int','cc', 'dist_int_nb_self'])
(gg.ggplot(dat_mod_stats.loc[fil].query(f'{V.COL_CONDNAME} in {C.fig2_cond}'),
           
           gg.aes(x=V.COL_IDCHANNAME, y=V.COL_R2))+
           gg.facet_grid(V.COL_CONDITION+'~.')+
              gg.geom_hline(gg.aes(yintercept=1))+
             gg.geom_hline(gg.aes(yintercept=0))+ 
           gg.geom_line(gg.aes(group=V.COL_IDCHANNAME), alpha=0.5)+
           gg.geom_point(gg.aes(color=V.COL_MODELCLASS_F), position=gg.position_dodge(width=0.9))+
           gg.expand_limits(y=1)+
            gg.ggtitle('Dist-to-rim vs neighbourhood')+
           gg.theme(axis_text_x = gg.element_text(angle = 90, hjust = 0.1), figure_size=(5,3)
            )
)

In [ ]:
dist_groups = {'Neighbourhood': ['dist', 'nb', 'dist_nb'],
               'Autocorrelation': ['dist_nb', 'self', 'dist_nb_self'],
               'Internal': ['int', 'dist_nb_self', 'dist_int_nb_self']}
COL_MODGROUP = 'model_groups'
COL_HIER_MODELCLASS = 'model'


dat_mod_hier = (pd.concat({k: dat_mod_stats.query(f'{V.COL_MODELCLASS} in {f}') for k, f in dist_groups.items()}
                         , names=[COL_MODGROUP,None]).reset_index(COL_MODGROUP, drop=False)
                .assign(**{COL_MODGROUP: lambda x: pd.Categorical(x[COL_MODGROUP], categories=dist_groups.keys())})
)

dat_mod_hier[V.COL_MODELCLASS] = pd.Categorical(dat_mod_hier[V.COL_MODELCLASS]
                                                ,categories=['dist',
                                                             'nb', 'dist_nb', 'self','dist_nb_self', 
                                                                                            'cc', 'int',
                                                                                      'dist_int_nb_self'])

dat_mod_hier[V.COL_MODELCLASS] = dat_mod_hier[V.COL_MODELCLASS].cat.rename_categories({'dist': 'global env',
                                                                                      'nb': 'nb markers',
                                                                                      'dist_nb': 'global env\n + nb markers',
                                                                                        'self': 'autocorr',
                                                                                                                                                                           
                                                                                      'dist_nb_self': 'global env\n + nb markers\n +autocorr',
                                                                                    # 'cc': 'cellcycle state',
                                                                                      'int': 'cell state',
                                                                                      'dist_int_nb_self': 'full model'}
                                                                                     )



In [ ]:
(gg.ggplot(dat_mod_hier.query(f'{V.COL_CONDNAME} in {C.main_conds}'),
           gg.aes(x=V.COL_IDCHANNAME, y=V.COL_R2))+
           gg.facet_grid(f'{COL_MODGROUP}~{V.COL_CONDNAME}')+
              gg.geom_hline(gg.aes(yintercept=1))+
             gg.geom_hline(gg.aes(yintercept=0))+ 
            # gg.geom_vline(gg.aes(xintercept=V.COL_IDCHANNAME))+ 
           #gg.geom_line(gg.aes(xgroup=V.COL_IDCHANNAME), alpha=0.5)+
             gg.geom_segment(gg.aes(y=V.COL_R2, xend=V.COL_IDCHANNAME,color=V.COL_MODELCLASS), yend=0,
                             position=gg.position_dodge(width=0.9)) +            
             #gg.geom_line(gg.aes(xgroup=V.COL_IDCHANNAME), alpha=0.5)+
           gg.geom_point(gg.aes(color=V.COL_MODELCLASS), position=gg.position_dodge(width=0.9))+
           gg.expand_limits(y=1)+
                 gg.ylab('R2')+
            gg.ggtitle('Dist-to-rim vs neighbourhood')+
         gg.scale_color_brewer(type='qual',palette='Dark2')+
           gg.theme(axis_text_x = gg.element_text(angle = 90, hjust = 0.1), figure_size=(15,5)
            )
)

In [ ]:
cmap_paper = ['#c957dbff', '#5784dbff', '#7570b3ff', '#57db94ff', '#66a61eff', '#b9db57ff', '#db5f57ff', '#666666ff']

In [ ]:
p=(gg.ggplot(dat_mod_hier.query(f'{V.COL_CONDNAME} in {C.fig2_cond}'),
           
           gg.aes(x=V.COL_IDCHANNAME, y=V.COL_R2))+
           gg.facet_grid(f'{COL_MODGROUP}~.')+
              gg.geom_hline(gg.aes(yintercept=1))+
             gg.geom_hline(gg.aes(yintercept=0))+ 
            # gg.geom_vline(gg.aes(xintercept=V.COL_IDCHANNAME))+ 
           #gg.geom_line(gg.aes(xgroup=V.COL_IDCHANNAME), alpha=0.5)+
             gg.geom_segment(gg.aes(y=V.COL_R2, xend=V.COL_IDCHANNAME,color=V.COL_MODELCLASS), yend=0,
                             position=gg.position_dodge(width=0.9)) +            
             #gg.geom_line(gg.aes(xgroup=V.COL_IDCHANNAME), alpha=0.5)+
           gg.geom_point(gg.aes(color=V.COL_MODELCLASS), position=gg.position_dodge(width=0.9))+
           gg.scale_y_continuous(limits=(0,1),expand=(0,0.01))+
                 gg.ylab('Variance Explained [R2]')+
   gg.xlab('')+
           # gg.ggtitle('Dist-to-rim vs neightbourhood')+
        gg.scale_color_manual(cmap_paper)+
   #gg.scale_color_manual(colorcet.glasbey_dark)+
           gg.theme(axis_text_x = gg.element_text(angle = -90, hjust = 0.1), 
                    figure_size=(4,5),
                    text=gg.element_text(size=6)
            )
)

fig = p.draw()

In [ ]:
fig.savefig(fol_out_paper/'subfig3_performance_submodels.pdf')
fig

In [ ]:
class MLAB:
    dist = '(global env)'
    dist_nb = '(global env \n local nb)'
    all_vs_dist = 'local nb\n vs\n global env'
    fullwodist_vs_fullnb = '(global env \n local nb)\n -\n (local nb)'
    fullwonb_vs_fullnb = '(global env \n local nb)\n -\n (global env)'
    auto = 'autocorr'
    dist_vs_fullnb = '(global env \n local nb \n autocorr)\n -\n (local nb \n autocorr)'
    auto_vs_distnb =  '(global env \n local nb \n autocorr)\n -\n (global env \n local nb)'
    auto_vs_fullnb = '(global env \n local nb \n autocorr)\n -\n (autocorr)'
    int_vs_fullnb = '(global env \n local nb \n autocorr)\n  -\n (cell state)'
    fullnb_vs_int = '(cell state)\n -\n (global env \n local nb \n autocorr)'
    nb = '(local nb)'
    int_vs_full = '(global env \n local nb \n autocorr \n cell state)\n -\n (cell state)'
    int = '(cell state)'
    fullnb = '(global env \n local nb \n autocorr)'
    int_vs_woint = '(global env \n local nb \n autocorr\n  cell state)\n -\n (global env \n local nb \n autocorr)'
    full = '(global env \n local nb \n autocorr \n int)'
model_comp = {MLAB.dist: f'{M.dist}',
              MLAB.nb: f'{M.nb}',
              MLAB.fullwodist_vs_fullnb: f'{M.dist_nb}-{M.nb}',
              MLAB.fullwonb_vs_fullnb: f'{M.dist_nb}-{M.dist}',
              
              MLAB.dist_nb: f'{M.dist_nb}',
              MLAB.auto: f'{M.self}',
              #MLAB.dist_vs_fullnb: f'{M.dist_nb_self}-{M.nb_self}',
              MLAB.auto_vs_fullnb: f'{M.dist_nb_self}-{M.self}',
              MLAB.auto_vs_distnb: f'{M.dist_nb_self}-{M.dist_nb}',

              
              MLAB.fullnb: f'{M.dist_nb_self}',
              MLAB.int: f'{M.int}',
              #MLAB.fullnb_vs_int: f'{M.int}-{M.dist_nb_self}',
               MLAB.int_vs_full: f'{M.dist_int_nb_self}-{M.int}',
              MLAB.int_vs_woint: f'{M.dist_int_nb_self}-{M.dist_nb_self}'
              #MLAB.full: f'{M.dist_int_nb_self}'
        }
COL_MODGROUP = 'model_groups'



dat_mod_comp = (pd.concat({k: dat_mod_stats_wide.eval(f'{V.COL_R2}={f}') for k, f in model_comp.items()}
                         , names=[COL_MODGROUP,None]).reset_index(COL_MODGROUP, drop=False)
                .assign(**{COL_MODGROUP: lambda x: pd.Categorical(x[COL_MODGROUP], categories=model_comp.keys())})
)

In [ ]:
with plt.style.context('ggplot'):
    sns.set_palette('deep')
    fig = plt.figure(figsize=(14,4))
    ax = sns.boxplot(x=COL_MODGROUP, y=V.COL_R2, data=dat_mod_comp, whis=0,fliersize=0, color='gray')
    for patch in ax.artists:
        r, g, b, a = patch.get_facecolor()
        patch.set_facecolor((r, g, b, .1))
    ax = sns.swarmplot(x=COL_MODGROUP, y=V.COL_R2,hue='is_delta',
                       data=dat_mod_comp.merge(dat_condmeta)
                           .assign(is_delta = lambda x: ['delta' if '-' in m else 'model' for m in x[COL_MODGROUP]]),
                       size=1.5, 
                      rasterized=True)
    ax.hlines(0,-20,20)
    plt.ylabel('(Delta) Variance explained [R2]')
    ax.tick_params(axis='x', labelsize=10)
    plt.xlabel('')
    ax.legend(loc='center left', bbox_to_anchor=(1., 0.5), ncol=1)
fig.savefig(fol_out_paper/'subfig3_delta_submodels_overall.pdf')

In [ ]:
p =(dat_mod_comp>>
 gg.ggplot(gg.aes(x=COL_MODGROUP, y=f'{V.COL_R2}'))+
  # gg.geom_hline(gg.aes(yintercept=1))+
 gg.geom_hline(gg.aes(yintercept=0))+
 gg.geom_boxplot()+
 # gg.geom_violin(alpha=0.4)+

 gg.geom_jitter(alpha=0.1, width=0.2, size=1)+
 gg.ylim((-0.25,0.75))+
 #gg.ylab('(R2_self)-(R2_distance-to-rim +nb)')+
             gg.xlab('')+
         gg.ylab('Δ(R2)')+
           gg.theme(axis_text_x = gg.element_text(angle = 0, hjust = 0.1, size=6), figure_size=(6,3)
            ))

p

In [ ]:
d = dat_mod_comp.loc[dat_mod_comp[COL_MODGROUP]  == MLAB.fullwodist_vs_fullnb]
print(np.mean(d[V.COL_R2]))
print(np.median(d[V.COL_R2]))
#print(np.mean(d[V.COL_R2] >=-0.001))
print(d[V.COL_R2].describe())
print(d[M.dist].describe())
d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2}>0.05').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)

In [ ]:
d = dat_mod_comp.loc[dat_mod_comp[COL_MODGROUP]  == MLAB.dist_vs_fullnb]
print(np.mean(d[V.COL_R2]))
print(np.median(d[V.COL_R2]))
print(np.mean(d[V.COL_R2] >=0))
print(d[V.COL_R2].describe())
print(d[M.dist].describe())
d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2}>0.05').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)

In [ ]:
d = dat_mod_comp.loc[dat_mod_comp[COL_MODGROUP]  == MLAB.fullwodist_vs_fullnb]
print(np.mean(d[V.COL_R2]))
print(np.median(d[V.COL_R2]))
print(d[V.COL_R2].describe())
print(d[M.dist].describe())
d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2}>0.05').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)

In [ ]:
d = dat_mod_comp.loc[dat_mod_comp[COL_MODGROUP]  == MLAB.auto_vs_fullnb]
print(np.mean(d[V.COL_R2]))
print(np.median(d[V.COL_R2]))
print(d[V.COL_R2].describe())
print(d[M.self].describe())
d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2} < -0.05').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)

In [ ]:
d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2}<-0.05').sort_values(V.COL_R2)

In [ ]:
d.eval(f'x={M.dist_nb}-{M.self}').sort_values('x')

In [ ]:
d = dat_mod_comp.loc[dat_mod_comp[COL_MODGROUP]  == MLAB.int_vs_fullnb]
print(np.mean(d[V.COL_R2]))
print(np.median(d[V.COL_R2]))
print(np.mean(d.sort_values(V.COL_R2, ascending=False)[V.COL_R2]<0))

d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2}<0').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)

In [ ]:
d = dat_mod_comp.loc[dat_mod_comp[COL_MODGROUP]  == MLAB.int_vs_full]
print(np.mean(d[V.COL_R2]))
print(np.median(d[V.COL_R2]))
print(np.mean(d.sort_values(V.COL_R2, ascending=False)[V.COL_R2]<0))
x=d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2}<-0.25').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)
print(x)
print(np.sum(x>=6))
x=d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2}<-0.1').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)
print(x)
print(np.sum(x>=6))
x=d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2}<-0.05').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)
print(x)
print(np.sum(x>=6))

In [ ]:
x=d.sort_values(V.COL_R2, ascending=False).query(f'{M.dist_nb_self}>0.25').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)
print(x)
print(np.sum(x>=6))

In [ ]:
x=d.sort_values(V.COL_R2, ascending=False).query(f'{M.dist_nb_self}>0.2').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)
print(x)
print(np.sum(x>=8))

In [ ]:
print(d.sort_values(V.COL_R2, ascending=False)[V.COL_R2].describe())
print(d.sort_values(M.dist_nb_self, ascending=False)[M.dist_nb_self].describe())

In [ ]:
np.mean(d.sort_values(V.COL_R2, ascending=False)[V.COL_R2] < -0.05)

In [ ]:
np.mean(d.sort_values(V.COL_R2, ascending=False)[M.dist_nb_self] > 0.2)

In [ ]:
d = dat_mod_comp.loc[dat_mod_comp[COL_MODGROUP]  == MLAB.int_vs_full]
print(np.mean(d[V.COL_R2]))
print(np.median(d[V.COL_R2]))
print(np.mean(d.sort_values(V.COL_R2, ascending=False)[V.COL_R2]<0))
d.sort_values(V.COL_R2, ascending=False).query(f'{M.dist_nb_self}>0.05').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)

In [ ]:
d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2}<0').head()

In [ ]:
(dat_mod_comp.loc[dat_mod_comp[COL_MODGROUP]  == MLAB.all_vs_dist]
 .query(f'{V.COL_R2}<0').sort_values([M.dist_nb_self], ascending=False).head()
)#.query(f'{M.dist_nb_self} > 0.1')

In [ ]:
(dat_mod_comp.loc[dat_mod_comp[COL_MODGROUP]  == MLAB.all_vs_dist]
 .query(f'{V.COL_R2}<0').sort_values([M.dist_nb_self], ascending=False)
 .groupby(V.COL_CHANNEL)[V.COL_CHANNEL].count()).head()

In [ ]:
(dat_mod_comp.loc[dat_mod_comp[COL_MODGROUP]  == MLAB.all_vs_dist]
 .query(f'{V.COL_R2}<0').sort_values([M.dist_nb_self], ascending=False)
 .groupby(V.COL_CHANNEL)[V.COL_CHANNEL].count()).head()

In [ ]:
dat = (dat_mod_comp.merge(dat_condmeta).groupby([COL_MODGROUP, V.COL_CELLLINE, V.COL_TP, V.COL_CHANNEL])[V.COL_R2].mean()).reset_index()

for g, d in dat[[COL_MODGROUP,V.COL_CHANNEL,V.COL_CELLLINE, V.COL_R2]].groupby(COL_MODGROUP):
    display(d.sort_values(V.COL_R2).head(5))
    display(d.sort_values(V.COL_R2, ascending=False).head(5))

In [ ]:

for g, d in (dat_mod_comp[[COL_MODGROUP,V.COL_CHANNEL, V.COL_R2]]
             .groupby([COL_MODGROUP, V.COL_CHANNEL])[V.COL_R2].mean().reset_index()
             .groupby(COL_MODGROUP)):
    display(d.sort_values(V.COL_R2).head())

In [ ]:

for g, d in dat_mod_comp[[COL_MODGROUP, 'channel',V.COL_CONDNAME, V.COL_R2]].groupby(COL_MODGROUP):
    display(d.sort_values(V.COL_R2).head(5))
    display(d.sort_values(V.COL_R2, ascending=False).head(5))

In [ ]:
dat_mod_stats_wide.eval(f'{M.self}-{M.dist_nb}').describe()

-> looks like this mostly holds - except cases where distance to rim + nb is super important, e.g. Ki67

In [ ]:
d = dat_mod_comp.loc[dat_mod_comp[COL_MODGROUP]  == MLAB.fullwodist_vs_fullnb]
print(np.mean(d[V.COL_R2]))
print(np.median(d[V.COL_R2]))
print(d[V.COL_R2].describe())
print(d[M.self].describe())
d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2} > 0.15').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)
d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2} > 0.1')

In [ ]:
d = dat_mod_comp.loc[dat_mod_comp[COL_MODGROUP]  == MLAB.fullwonb_vs_fullnb]
print(np.mean(d[V.COL_R2]))
print(np.median(d[V.COL_R2]))
print(d[V.COL_R2].describe())
print(d[M.self].describe())
d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2} > 0.15').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)
d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2} > 0.1')

In [ ]:
d = dat_mod_comp.loc[dat_mod_comp[COL_MODGROUP]  == MLAB.auto_vs_distnb]
print(np.mean(d[V.COL_R2]))
print(np.median(d[V.COL_R2]))
print(d[V.COL_R2].describe())
print(d[M.self].describe())
d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2} > 0.15').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)
d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2} > 0.1')

In [ ]:
d = dat_mod_comp.loc[dat_mod_comp[COL_MODGROUP]  == MLAB.int_vs_full]
print(np.mean(d[V.COL_R2]))
print(np.median(d[V.COL_R2]))
print(d[V.COL_R2].describe())
print(d[M.self].describe())
d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2} > 0.15').groupby(V.COL_CHANNEL).size().sort_values(ascending=False)
d.sort_values(V.COL_R2, ascending=False).query(f'{V.COL_R2} > 0.15')[V.COL_CHANNEL].value_counts()

Make a stacked barplot with models according to the hierarchy

In [ ]:
mod_tup = [(M.dist, 'global env'), 
        (M.dist_nb, ' +local nb'),
        (M.dist_nb_self, ' +autocorr'),
        ('dist_nb_self_cc', ' +cellcycle'),
        (M.dist_int_nb_self, '+cell state')]
mods = [m[0] for m in mod_tup]
mod_lab = dict(mod_tup) 

In [ ]:
dat_mod_stats[V.COL_MODELCLASS+'2'] = pd.Categorical(dat_mod_stats[V.COL_MODELCLASS], categories=mods)

In [ ]:
fil = dat_mod_stats[V.COL_MODELCLASS].isin(mods)
dat_mod_stats[V.COL_MODELCLASS+'2'] = pd.Categorical(
    dat_mod_stats[V.COL_MODELCLASS], categories=reversed(mods)
    ).rename_categories(mod_lab)



tdat = dat_mod_stats.loc[fil].sort_values(V.COL_MODELCLASS+'2')

In [ ]:
def tofac(s):
    return pd.Categorical(s)

In [ ]:
"""
Aim: Make a plot like the one above with matplotlib
- Add small triangles on the bottom axis to indicate increasing size
- colorcode the two timepoints
- make the bars of consecutive sizes touching, add dashed line between timepoints
- Separate markers by a small offset
- plot every cellline as separate subplot
"""
from functools import reduce # Valid in Python 2.6+, required in Python 3
import operator
# 1) calculate x positions

def make_category(col, categories=None):
    if col.dtype.name != 'category':
        if categories is None:
            categories = sorted(np.unique(col))
        return pd.Categorical((col),categories=categories)
    else:
        return col
    
def get_x_pos(datord,offsets):
    """
    Gets x position the order of variables calculated by:
        sum(order(var)*offset_var for var in datord)
    Args:
        datord: an dataframe with the variables usef for ordering
        off: a vector with offsets per n_variable
    Returns:
        Postion
    """
    assert(datord.shape[1] == len(offsets))
    datord = datord.apply(make_category, axis=0)
    
    return np.sum([get_uniord(datord[col].cat.codes)*off for col, off in zip(list(datord), offsets)], axis=0)


def get_uniord(vals):
    """
    Gets replaces the values by the order of the unique values
    """
    univals = vals.unique()
    orderdic = {val: order for order, val in enumerate(sorted(univals))}
    return vals.replace(orderdic)
        
def get_offsets(datord):
    ncounts = datord.nunique()
    offsets = [1] + [ reduce(operator.mul, ncounts[:(i+1)]) for i in range(len(ncounts)-1)]
    return offsets
    

def add_triangle_annotations(dat, xgrpvars, xvar, ax, w_scale=0.9, y_offset=-0.01, h=0.075, colors=None):
    if colors is None:
        cols = plt.cm.Accent(range(10))
    dat = dat.groupby(xgrpvars[1:]).agg({xvar: 'mean',
                                                         xgrpvars[0]: 'nunique'}).rename(columns={xgrpvars[0]: 'n_width'}).reset_index()
    dat[xgrpvars[1]] = make_category(dat[xgrpvars[1]])

    for idx, row in dat.iterrows():
        c= cols[dat.loc[[idx],xgrpvars[1]].cat.codes.values[0]]
        x=row[xvar]
        w=w_scale*row['n_width']
        ax.add_patch(plt.Polygon(((x-w/2,y_offset),(x+w/2,y_offset),(x+w/2, y_offset-h)),
                                   fill=True,clip_on=False, color=c))
        
def add_channel_annotations(dat, xvar, labelvar, ax, w_scale=0.9, y_offset=-0.01, h=0.075, colors=None):
    if colors is None:
        cols = plt.cm.Accent(range(10))
    dat = dat.groupby(labelvar)[xvar].mean().reset_index()
    lab_pos = dat[xvar]
    lab_text = dat[labelvar]
    ax.get_xaxis().set_ticks(lab_pos)
    ax.get_xaxis().set_ticklabels(lab_text)
    ax.xaxis.labelpad = 20



In [ ]:
testdat = (tdat.merge(dat_condmeta)
           #.query(f'{V.COL_CELLLINE} == "293T"')
           #.query(f'{V.COL_MODELCLASS} =="dist_int_nb_self"')
         
          )

In [ ]:
%matplotlib inline

In [ ]:
from mizani.palettes import hue_pal

In [ ]:
col_mclass = V.COL_MODELCLASS+'2'

In [ ]:
ncols = len(np.unique(testdat[col_mclass]))
colors_hue = hue_pal(h=.01, l=.6, s=.65, color_space='hls')
col_map = list((colors_hue(ncols)))

In [ ]:

#dat = dat.sort_values(col_mclass)



xgrpvars = [V.COL_CONC, V.COL_TP,  V.COL_IDCHANNAME]
ygrpvar = V.COL_CELLLINE

def do_r2_barplot(dat, # the dataframe
                  y_var,
                  xgrpvars,
                  ygrpvars,
                  col_mclass,
                  figsize=(15,5),
                  group_offsets=None,
                  col_map=None,
                  triangle_annotations=True,
                  channel_annotations=True,
                  legend=True,
                  bar_width=1,
                  legend_anchor=(1.05, 0.5),
                  dpi=600,
                  subplot_label_offset=5,
                  channel_text_size=12,
                  channel_rotation=-90,
                  labelvar=None
                 ):

    ncols = len(np.unique(dat[col_mclass]))
    if col_map is None:
        colors_hue = hue_pal(h=.01, l=.6, s=.65, color_space='hls')
        col_map = list((colors_hue(ncols)))
    if group_offsets is None:
        group_offsets =[0,0.3,2]

    offsets = get_offsets(dat[xgrpvars])
    offsets = [o+go for o, go in zip(offsets, group_offsets)]

    XVAR = 'xpos'
    dat[XVAR] = get_x_pos(dat[xgrpvars], offsets)
    n_rows = dat[ygrpvars].nunique()
    fig, axs = plt.subplots(n_rows,1,figsize=figsize, dpi=dpi)
    try:
        len(axs)
    except:
        axs = [axs]

    for (ylab, dat), ax in zip(dat.groupby(ygrpvars), axs):
        dat = dat.sort_values(col_mclass,ascending=True)
        c = [col_map[i] for i in dat[col_mclass].cat.codes.values]
        ax.bar(np.unique(dat[XVAR]), [1 for i in range(dat[XVAR].nunique())], color='grey', width=bar_width)
        ax.bar(dat[XVAR].values, dat[y_var].values, width=bar_width, color=c)
        # set ticks where your images will be
        ax.get_xaxis().set_ticks([])
        # remove tick labels
        ax.get_xaxis().set_ticklabels([])
        ax.margins(x=0.01,y=0)
        ax.text(dat[XVAR].max()+subplot_label_offset, 0.5, ylab)

    if channel_annotations:
        if labelvar is None:
            labelvar = xgrpvars[-1]
        add_channel_annotations(dat, xvar=XVAR, labelvar=labelvar, ax=ax)

        for tick in ax.get_xticklabels():
            tick.set_rotation(channel_rotation)
            tick.set_size(channel_text_size)
            tick.set_ha('left')
        ax.xaxis.set_tick_params(length=0,pad=10)

    if triangle_annotations:
        add_triangle_annotations(dat, xgrpvars, XVAR, ax, w_scale=0.9, y_offset=-0.01, h=0.12, colors=None)

    # xlabel
    ax2 = fig.add_subplot(111, frameon=False)
    ax2.set_ylabel('Variance Explained [R2]')
    ax2.get_xaxis().set_ticks([])
    ax2.get_xaxis().set_ticklabels([])
    ax2.get_yaxis().set_ticks([])
    ax2.get_yaxis().set_ticklabels([])
    ax2.yaxis.labelpad = 40

    if legend:
        # Put a legend to the right of the current axis
        lab_class = dat[col_mclass].cat.categories
        lab_cols = col_map
        lab_patches =[mpatches.Patch(color=c, label=lab) for c, lab in zip(lab_cols, lab_class)]

        ax2.legend(handles=lab_patches, loc='center left', bbox_to_anchor=legend_anchor)

    return fig, dat

In [ ]:
with plt.rc_context({'font.size': 6}):
    fig, dat = do_r2_barplot(tdat.merge(dat_condmeta),
                             r_var,
                             xgrpvars=[V.COL_CONC, V.COL_TP,  V.COL_IDCHANNAME],
                  ygrpvars=V.COL_CELLLINE,
                             col_mclass=V.COL_MODELCLASS+'2',
                                              channel_text_size=5,
                             figsize=(8,2),
                  channel_rotation=-90)
    for ax in fig.axes:
        ax.set_yticks([0, 0.5, 1])
fig.savefig(fol_out_paper/'fig4_overview_variability_explained.pdf')
fig

# Compare pErk

In [ ]:
t = (dat_mod_stats.merge(dat_condmeta).merge(dat_pannel_ord, left_on=V.COL_CHANNELNAME, right_on=V.COL_METAL, how='left')
 .rename(columns={'class': 'mark_class'})
 .query('mark_class== "egf"')
 #.query(f'{V.COL_CONDNAME} in {C.main_conds}')
 .pivot_table(index=[V.COL_IDCHANNAME,V.COL_CHANNELNAME,  V.COL_CELLLINE],columns=[V.COL_MODELCLASS_F], values=V.COL_R2, aggfunc=np.mean)
)
t.columns = t.columns.astype('str')


t= (t.assign(**{'full_vs_fulnb': lambda x: x['dist_nb_self']/x['dist_int_nb_self'],
               'full_vs_self': lambda x: x['self']/x['dist_int_nb_self'],
            'full_vs_dist': lambda x: x['dist']/x['dist_int_nb_self'],
            })
)

In [ ]:
t

In [ ]:
(dat_mod_stats.merge(dat_condmeta).merge(dat_pannel_ord, left_on=V.COL_CHANNELNAME, right_on=V.COL_METAL, how='left')
 .rename(columns={'class': 'mark_class'})
 .query('mark_class== "egf"')
 .query(f'{V.COL_CONDNAME} in {C.main_conds}')
 .pivot_table(index=[V.COL_IDCHANNAME,  V.COL_CELLLINE],columns=[V.COL_MODELCLASS_F], values=V.COL_R2)

)

In [ ]:
with plt.rc_context({'font.size': 6}):
    fig, dat = do_r2_barplot(tdat.merge(dat_condmeta).groupby([V.COL_CELLLINE, V.COL_MODELCLASS+'2'])[r_var]
                         .mean().reset_index().assign(x=1),
                         r_var,
                         xgrpvars=[ V.COL_CELLLINE],
                         
              ygrpvars='x',
                         group_offsets =[0],
                         triangle_annotations=False,
                         channel_annotations=True,
                         col_mclass=V.COL_MODELCLASS+'2',
                            bar_width=0.9,
                        legend_anchor=(1.,0.5),
                             channel_text_size=5,
                             figsize=(1.7,1.7),
                  channel_rotation=0)

ax = fig.axes[0]
ax.yaxis.set_ticks(np.arange(0, 1.2, 0.2))
fig.savefig(fol_out_paper / 'overview_allcond.pdf')
fig

In [ ]:
fig, dat = do_r2_barplot(tdat.merge(dat_condmeta).merge(dat_med.query(f'{V.COL_VALUE} > (1/2**16)')).groupby([V.COL_TP, V.COL_CONC,V.COL_CELLLINE, V.COL_MODELCLASS+'2'])[r_var]
                         .mean().reset_index().assign(x=1),
                         r_var,
                         xgrpvars=[V.COL_CONC, V.COL_TP, V.COL_CELLLINE],
                         
              ygrpvars='x',
                        # group_offsets =[0],
                         triangle_annotations=True,
                         channel_annotations=True,
                         col_mclass=V.COL_MODELCLASS+'2',
                            bar_width=0.9,
                        figsize=(7,5),
                        legend_anchor=(1.1,0.5),
                        subplot_label_offset=1,
                        channel_text_size=10)

In [ ]:
fig, dat = do_r2_barplot(tdat.merge(dat_condmeta).groupby([V.COL_TP, V.COL_CONC,V.COL_CELLLINE, V.COL_MODELCLASS+'2'])[r_var]
                         .mean().reset_index().assign(x=1),
                         r_var,
                         xgrpvars=[V.COL_CONC, V.COL_TP, V.COL_CELLLINE],
                         
              ygrpvars='x',
                        # group_offsets =[0],
                         triangle_annotations=True,
                         channel_annotations=True,
                         col_mclass=V.COL_MODELCLASS+'2',
                            bar_width=0.9,
                        figsize=(7,5),
                        legend_anchor=(1.1,0.5),
                        subplot_label_offset=1,
                        channel_text_size=10)

Do this plot filtered for markers with > log(1) median count

In [ ]:
fig, dat = do_r2_barplot(tdat.merge(dat_condmeta).groupby([V.COL_CELLLINE, V.COL_MODELCLASS+'2', V.COL_MODELCLASS])[r_var]
                         .mean().reset_index().assign(x=1),
                         r_var,
                         xgrpvars=[V.COL_CELLLINE],
                         
              ygrpvars='x',
                        # group_offsets =[0],
                         triangle_annotations=False,
                         channel_annotations=True,
                         col_mclass=V.COL_MODELCLASS+'2',
                            bar_width=0.9,
                        figsize=(7,5),
                        legend_anchor=(1.1,0.5),
                        subplot_label_offset=1,
                        channel_text_size=10)

In [ ]:
d = (tdat.merge(dat_condmeta).groupby([V.COL_CELLLINE, V.COL_MODELCLASS+'2', V.COL_MODELCLASS])[r_var]
                         .mean().reset_index().assign(x=1))
fig, dat = do_r2_barplot(d,
                         r_var,
                         xgrpvars=[V.COL_CELLLINE],
                         
              ygrpvars='x',
                        # group_offsets =[0],
                         triangle_annotations=False,
                         channel_annotations=True,
                         col_mclass=V.COL_MODELCLASS+'2',
                            bar_width=0.9,
                        figsize=(3,2),
                        legend_anchor=(1.1,0.5),
                        subplot_label_offset=1,
                        channel_text_size=5)
fig.show()
display(d.groupby([V.COL_MODELCLASS+'2'])[V.COL_R2].describe())

In [ ]:

def c_stats(x):
    xnb = x.query(f'{V.COL_MODELCLASS} == "dist_nb_self"')[V.COL_R2].values[0]
    xfull = x.query(f'{V.COL_MODELCLASS} == "dist_int_nb_self"')[V.COL_R2].values[0]
    xcc = x.query(f'{V.COL_MODELCLASS} == "dist_nb_self_cc"')[V.COL_R2].values[0]
    xdist = x.query(f'{V.COL_MODELCLASS} == "dist"')[V.COL_R2].values[0]
    return pd.Series({'full': xfull,
                      'nb': xnb,
                      'frac_int': xnb/xfull,
                     'eff_cc': xcc-xnb,
                     'frac_dist': xdist/xnb})



# overll
d = (tdat.merge(dat_condmeta).groupby([V.COL_CELLLINE, V.COL_MODELCLASS+'2', V.COL_MODELCLASS])[r_var]
                         .mean().reset_index().assign(x=1))
dstat = d.groupby([V.COL_CELLLINE]).apply(c_stats)
display(dstat)
display(dstat.describe())
# strong markers only
d = (tdat.merge(dat_condmeta).merge(dat_med.query(f'{V.COL_VALUE} > (1/2**16)')).groupby([V.COL_CELLLINE, V.COL_MODELCLASS+'2', V.COL_MODELCLASS])[r_var]
                         .mean().reset_index().assign(x=1))
dstat = d.groupby([V.COL_CELLLINE]).apply(c_stats)
display(dstat)
display(dstat.describe())

# all conditions
# strong markers only
d = (tdat.merge(dat_condmeta).merge(dat_med.query(f'{V.COL_VALUE} > -(1/2**16)')).groupby([V.COL_CELLLINE, V.COL_CONDNAME, V.COL_MODELCLASS+'2', V.COL_MODELCLASS])[r_var]
                         .mean().reset_index().assign(x=1))
dstat = d.groupby([V.COL_CONDNAME]).apply(c_stats)
display(dstat)
display(dstat.describe())

Calculate the coefficient of variation of all the submodels

In [ ]:
# all conditions
# strong markers only
import scipy.stats

d = (tdat.merge(dat_condmeta)
     #.merge(dat_med.query(f'{V.COL_VALUE} > (1/2**16)'))
     .groupby([V.COL_CELLLINE,  V.COL_MODELCLASS+'2', V.COL_MODELCLASS, V.COL_CHANNELNAME])[V.COL_R2]
    .agg(**{'sd': np.std, 'mn': np.mean})
    .groupby([V.COL_CELLLINE,  V.COL_MODELCLASS+'2', V.COL_MODELCLASS])
        .apply(lambda x: np.mean(x['sd']))
    )
display(d)
#display(d.groupby([ V.COL_MODELCLASS+'2', V.COL_MODELCLASS]).describe())
display(d.groupby([ V.COL_MODELCLASS+'2', V.COL_MODELCLASS]).describe())

In [ ]:

d = (tdat.merge(dat_condmeta)
     #.merge(dat_med.query(f'{V.COL_VALUE} > (1/2**16)'))
     .groupby([V.COL_CELLLINE,  V.COL_MODELCLASS+'2', V.COL_MODELCLASS,
               V.COL_CHANNELNAME])[V.COL_R2]
     .mean()
     .groupby([V.COL_MODELCLASS+'2', V.COL_MODELCLASS, V.COL_CHANNELNAME])
    .agg(**{'sd': np.std, 'mn': np.mean})
    .groupby([V.COL_MODELCLASS+'2', V.COL_MODELCLASS])
        .apply(lambda x: np.mean(x['sd']))
    )
display(d)

In [ ]:
# all conditions
# strong markers only.query(f'{V.COL_CONDNAME} in {C.main_conds}'),


d = (tdat.merge(dat_condmeta)
     #.merge(dat_med.query(f'{V.COL_VALUE} > (1/2**16)'))
     .groupby([V.COL_CELLLINE,  V.COL_MODELCLASS+'2', V.COL_MODELCLASS, V.COL_CHANNELNAME])[V.COL_R2].mean()
  .groupby([ V.COL_MODELCLASS+'2', V.COL_MODELCLASS, V.COL_CHANNELNAME]).apply(np.std)
         .groupby([ V.COL_MODELCLASS+'2', V.COL_MODELCLASS ]).describe())
display(d)

d = (tdat.merge(dat_condmeta)
     .query(f'{V.COL_CONDNAME} in {C.main_conds}')
    # .merge(dat_med.query(f'{V.COL_VALUE} > (1/2**16)'))
     .groupby([V.COL_CELLLINE,  V.COL_MODELCLASS+'2', V.COL_MODELCLASS, V.COL_CHANNELNAME])[V.COL_R2].median()
  .groupby([ V.COL_MODELCLASS+'2', V.COL_MODELCLASS, V.COL_CHANNELNAME]).apply(np.std)
         .groupby([ V.COL_MODELCLASS+'2', V.COL_MODELCLASS ]).describe())
display(d)

In [ ]:
fig, dat = do_r2_barplot(tdat.merge(dat_condmeta).groupby([V.COL_TP, V.COL_CONC,V.COL_CELLLINE, V.COL_MODELCLASS+'2'])
                         .mean().reset_index().assign(x=1),
                         r_var,
                         xgrpvars=[V.COL_CONC, V.COL_TP, V.COL_CELLLINE],
                         
              ygrpvars='x',
                        # group_offsets =[0],
                         triangle_annotations=True,
                         channel_annotations=True,
                         col_mclass=V.COL_MODELCLASS+'2',
                            bar_width=0.9,
                        figsize=(7,5),
                        legend_anchor=(1.1,0.5),
                        subplot_label_offset=1,
                        channel_text_size=10)

In [ ]:
def get_stats(x):
    return pd.Series({'dist': 0,
       'dist_nb': x['dist_nb']-x['nb'],
       'dist_nb_self':  x['dist_nb_self']-x['self'],
       'dist_nb_self_cc': x['dist_nb_self_cc']-x['cc'],
       'dist_int_nb_self':  x['dist_int_nb_self']-x['int']})

def get_delta_dat(dat):
    d_wide = dat.pivot_table(V.COL_R2, [V.COL_CHANNELNAME, V.COL_CONDNAME],V.COL_MODELCLASS)
    return d_wide
    #d_wide.apply(, axis=0)
    

curdat = dat_mod_stats

d=get_delta_dat(curdat)
d = d.apply(get_stats, axis=1)
d.columns.name = V.COL_MODELCLASS
d = d.stack()
d.name = 'delta'
d=d.reset_index(drop=False)
curdat = curdat.merge(d)
V.COL_TEST = 'test'
curdat[V.COL_TEST] = pd.Categorical(curdat[V.COL_MODELCLASS], categories=['dist', 'nb','dist_nb', 'self', 'dist_nb_self', 'cc', 'dist_nb_self_cc', 'int', 'dist_int_nb_self'])
curdat[V.COL_MODELCLASS+'2'] = pd.Categorical(
    curdat[V.COL_MODELCLASS], categories=reversed(mods)
    ).rename_categories(mod_lab)

(gg.ggplot(curdat.query(f'({V.COL_CONDNAME} in {C.main_conds})'), gg.aes(x=V.COL_TEST))+
     gg.facet_grid(f'{V.COL_GOODNAME}~{V.COL_CONDNAME}')+
    gg.geom_bar(gg.aes( y=V.COL_R2, fill=V.COL_MODELCLASS+'2'),stat='identity')+
    gg.geom_bar(gg.aes( y='delta'),stat='identity')+
    gg.theme(figure_size=(5,30))
)



In [ ]:
(gg.ggplot(curdat.query(f'({V.COL_CONDNAME} in {C.main_conds})'), gg.aes(x=V.COL_TEST))+
     gg.facet_grid(f'{V.COL_GOODNAME}~{V.COL_CONDNAME}')+
    gg.geom_bar(gg.aes( y=f'{V.COL_R2}-delta', fill=V.COL_MODELCLASS+'2'),stat='identity')+
    gg.theme(figure_size=(5,30))
)

In [ ]:
def get_stats(x):
    return pd.Series({'dist': 0,
       'dist_nb': x['dist_nb']-x['nb'],
       'dist_nb_self':  x['dist_nb_self']-x['self'],
       'dist_nb_self_cc': x['dist_nb_self_cc']-x['cc'],
       'dist_int_nb_self':  x['dist_int_nb_self']-x['int']})

def get_delta_dat(dat):
    d_wide = dat.pivot_table(V.COL_R2, [V.COL_CHANNELNAME, V.COL_CONDNAME],V.COL_MODELCLASS)
    return d_wide
    #d_wide.apply(, axis=0)
    

curdat = dat_mod_stats

d=get_delta_dat(curdat)
d = d.apply(get_stats, axis=1)
d.columns.name = V.COL_MODELCLASS
d = d.stack()
d.name = 'delta'
d=d.reset_index(drop=False)
curdat = curdat.merge(d)
V.COL_TEST = 'test'
curdat[V.COL_TEST] = pd.Categorical(curdat[V.COL_MODELCLASS], categories=['dist', 'nb','dist_nb', 'self', 'dist_nb_self', 'cc', 'dist_nb_self_cc', 'int', 'dist_int_nb_self'])
curdat[V.COL_MODELCLASS+'2'] = pd.Categorical(
    curdat[V.COL_MODELCLASS], categories=reversed(mods)
    ).rename_categories(mod_lab)

(gg.ggplot(curdat.query(f'({V.COL_CONDNAME} in {C.main_conds})'), gg.aes(x=V.COL_TEST))+
     gg.facet_grid(f'{V.COL_GOODNAME}~{V.COL_CONDNAME}')+
    gg.geom_bar(gg.aes( y=V.COL_R2, fill=V.COL_MODELCLASS+'2'),stat='identity')+
    gg.geom_bar(gg.aes( y='delta'),stat='identity')+
    gg.theme(figure_size=(5,30))
)

In [ ]:
p=(curdat.groupby([V.COL_MODELCLASS + '2',V.COL_TEST])[[V.COL_R2,'delta']].mean().reset_index()
 >>
gg.ggplot(gg.aes(x=1, y=r_var))+
           #gg.facet_grid('.~'+V.COL_IDCHANNAME, scales='free_x')+
           #gg.facet_grid(V.COL_CONDITION+'~.')+
              gg.geom_hline(gg.aes(yintercept=1))+
             gg.geom_hline(gg.aes(yintercept=0))+ 
          
           
 #gg.geom_line(gg.aes(group=V.COL_IDCHANNAME), position=gg.position_dodge(width=0.2), alpha=0.5)+
             #gg.geom_segment(gg.aes(xend=V.COL_IDCHANNAME, y=0, yend=V.COL_R2, group=V.COL_IDCHANNAME))+
             gg.geom_bar(gg.aes(y=1 ), fill='grey', stat="identity",position = "identity")+
  gg.geom_bar(gg.aes(fill=V.COL_MODELCLASS+'2'), stat="identity",position = "identity")+
           gg.expand_limits(y=1)+
            gg.ggtitle('Additional variance explained by extended models')+
           gg.theme(axis_text_x = gg.element_text(angle = 90, hjust = 0.1), figure_size=(5,6)
                   )           
  )
gg.ggsave(p, fol_out_paper / 'bar_concept.pdf')
p

In [ ]:
#def full(**kwargs):
#    return 'full'
full='full'
p=(gg.ggplot(curdat.groupby([V.COL_MODELCLASS + '2',V.COL_TEST])[[V.COL_R2,'delta']].mean().reset_index() , gg.aes(x=V.COL_TEST))+
    # gg.facet_grid(f'{V.COL_GOODNAME}~{V.COL_CONDNAME}')+
    gg.geom_bar(gg.aes( y=V.COL_R2, fill=V.COL_MODELCLASS+'2'),stat='identity')+
    gg.geom_bar(gg.aes( y='delta'),stat='identity',fill='grey',)+
 
    gg.geom_bar(gg.aes(y=1,x='full' ), fill='grey', stat="identity",position = "identity")+
      gg.geom_bar(gg.aes(x='full', y=V.COL_R2, fill=V.COL_MODELCLASS+'2'), stat="identity",position = "identity")+
   
      gg.theme(figure_size=(5,5))

)
gg.ggsave(p, fol_out_paper / 'bar_concept.pdf')
p

In [ ]:
curdat[V.COL_TEST].cat.remove_unused_categories()

In [ ]:
width_bar = 0.6

fillvar = 'fillvar'
x_var = 'xvar'

fill_labels = {'cell state': 'Full cell state',
              'cellcycle': 'Cell cycle markers',
              'autocorr': 'Autocorrelation',
              'local nb': 'Neighbourhood',
              'global env': 'Environment'}

dtfill = curdat[V.COL_MODELCLASS + '2'].dtype
curdat[fillvar] = (curdat[V.COL_MODELCLASS + '2'].cat
                   .rename_categories({c: c.replace('+','').strip() for c in dtfill.categories}))
curdat[x_var] = (curdat[V.COL_MODELCLASS+'2'].cat.set_categories(reversed(dtfill.categories)))

tdat = (curdat
   .groupby(list(set([fillvar, x_var])),observed=True)[[V.COL_R2,'delta']]
   .mean()
      .reset_index()
   .assign(**{x_var: lambda x: x[x_var].cat.remove_unused_categories()
                                     .cat.add_categories(['full'])
                 }))
        
dt = tdat[x_var].dtype
tdat[fillvar] = tdat[fillvar].cat.rename_categories(fill_labels)
p= (tdat.sort_values(by=x_var, ascending=False)

   >>
    gg.ggplot(
             gg.aes(x=x_var))
    # gg.facet_grid(f'{V.COL_GOODNAME}~{V.COL_CONDNAME}')+
   + gg.geom_bar(gg.aes( y=V.COL_R2, fill=fillvar),stat='identity', width=width_bar)
   + gg.geom_bar(gg.aes( y='delta'),stat='identity',fill='lightgrey', width=width_bar)
 
   + gg.geom_bar(gg.aes(y=1,x='dt.categories[-1]' ), fill='grey', stat="identity",position = "identity", width=width_bar)
   +   gg.geom_bar(gg.aes(x='dt.categories[-1]', y=V.COL_R2, fill=fillvar),
                  stat="identity",position = "identity", width=width_bar)
   # gg.geom_segment(gg.aes(x=f'({x_var}.cat.codes)+1-{width_bar}/2', xend=f'len(dt.categories)+{width_bar}/2',
   #                        y=V.COL_R2, yend=V.COL_R2), linetype=':')+
   + gg.geom_segment(gg.aes(x=f'({x_var}.cat.codes)+1-{width_bar}/2', xend=f'({x_var}.cat.codes)+2-{width_bar}/2',
                          y=V.COL_R2, yend=V.COL_R2), linetype='-')
       + gg.geom_segment(gg.aes(x=f'({x_var}.cat.codes)+1-{width_bar}/2', xend=f'({x_var}.cat.codes)+1+{width_bar}/2',
                          y='delta', yend='delta'), linetype='-', color='grey')
    
           + gg.geom_segment(gg.aes(x=f'(len(dt.categories))-{width_bar}/2', xend=f'(len(dt.categories))+{width_bar}/2',
                          y=V.COL_R2, yend=V.COL_R2), linetype='-', color='grey')
   +    gg.geom_hline(yintercept=0)
    
   # + gg.scale_fill_manual(col_map)
       #gg.geom_hline(yintercept=0.5,linetype='--')+
   +    gg.xlab('')
   +  gg.ylab('Variability Explained [R2]')
   + gg.labs(fill='Modules')
         + gg.theme_bw()
   +   gg.theme(figure_size=(2.7,2),
              axis_text_x=gg.element_text(size=5),
              legend_text=gg.element_text(ha='left'),
               text=gg.element_text(size=5))
   + gg.coord_cartesian(ylim=(0,0.5))

    + gg.scale_fill_manual(col_map,guide = gg.guide_legend(reverse = False),
                    )
    

)
#gg.ggsave(p, fol_out_paper / 'bar_concept.pdf')
p

In [ ]:
gg.ggsave(p, fol_out_paper / 'bar_concept_1.pdf')

In [ ]:
fol_out_paper

In [ ]:
tdat = (curdat
   .groupby([V.COL_MODELCLASS + '2',V.COL_TEST],observed=True)[[V.COL_R2,'delta']]
   .mean()
      .reset_index()
   .assign(**{V.COL_TEST: lambda x: x[V.COL_TEST].cat.remove_unused_categories()
                                     .cat.add_categories(['full'])
                 })
       )
dt = tdat[V.COL_TEST].dtype

In [ ]:
c=tdat[V.COL_TEST].cat
tdat[V.COL_TEST].cat.codes

In [ ]:
p=(gg.ggplot(curdat.groupby([V.COL_MODELCLASS + '2',V.COL_TEST])[[V.COL_R2,'delta']].mean().reset_index() , gg.aes(x=V.COL_TEST))+
    # gg.facet_grid(f'{V.COL_GOODNAME}~{V.COL_CONDNAME}')+
    gg.geom_bar(gg.aes( y=V.COL_R2, fill=V.COL_MODELCLASS+'2'),stat='identity')+
    gg.geom_bar(gg.aes( y='delta'),stat='identity')+
    gg.theme(figure_size=(5,5))
)
gg.ggsave(p, fol_out_paper / 'bar_concept.pdf')
p

In [ ]:
(gg.ggplot(curdat.groupby([V.COL_MODELCLASS + '2',V.COL_TEST, V.COL_CONDNAME])[[V.COL_R2]].mean().reset_index().merge(dat_condmeta) ,
           gg.aes(x=V.COL_CONC))+
    gg.facet_grid(f'{V.COL_CELLLINE}~{V.COL_TP}+{V.COL_TEST}')+
    gg.geom_bar(gg.aes( y=V.COL_R2, fill=V.COL_MODELCLASS+'2'),stat='identity')+
    #gg.geom_bar(gg.aes( y='delta'),stat='identity')+
    gg.theme(figure_size=(5,5))
)

In [ ]:
(gg.ggplot(curdat.groupby([V.COL_MODELCLASS + '2',V.COL_TEST])[[V.COL_R2,'delta']].mean().reset_index() , gg.aes(x=V.COL_TEST))+
    # gg.facet_grid(f'{V.COL_GOODNAME}~{V.COL_CONDNAME}')+
    gg.geom_bar(gg.aes( y=V.COL_R2, fill=V.COL_MODELCLASS+'2'),stat='identity')+
    gg.geom_bar(gg.aes( y='delta'),stat='identity')+
    gg.theme(figure_size=(5,5))
)

In [ ]:
(gg.ggplot(curdat.groupby([V.COL_MODELCLASS + '2',V.COL_TEST])[[V.COL_R2,'delta']].mean().reset_index() ,
           gg.aes(x='"all"'))+
    # gg.facet_grid(f'{V.COL_GOODNAME}~{V.COL_CONDNAME}')+
    gg.geom_bar(gg.aes( y=V.COL_R2, fill=V.COL_MODELCLASS+'2'),stat='identity', position = "identity")+
    #gg.geom_bar(gg.aes( y='delta'),stat='identity')+
    gg.theme(figure_size=(1,5))+
     gg.coord_cartesian(ylim=(0,1))
)

In [ ]:
import seaborn as sns
from pandas.api.types import CategoricalDtype
from math import pi

In [ ]:
def make_spider( x, y, present_categories = True, color=None, xsize=14, **kwargs):
    ax = plt.gca()
    
    
    y_dic = {c: val for c, val in zip(x,y)}
    # number of variable
    all_categories = x.unique()
    if present_categories:
        categories= [c for c in x.values.categories if c in all_categories]
    N = len(categories)

    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles_dic = {var: n / float(N) * 2 * pi for n, var in enumerate(categories)}
    angles = [angles_dic[c] for c in categories]
    angles += angles[:1]

    # If you want the first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)

    # Draw one axe per variable + add labels labels yet
    plt.xticks(angles[:-1], categories)

    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([0.25,0.5,0.75,1], size=0)
    plt.ylim(0,1)

    # Ind1
    values= [y_dic.get(c,0) for c in categories]
    values += values[:1]
    ax.plot(angles, values, color=color, linewidth=2, linestyle='solid')
    #ax.fill(angles, values, color=color, alpha=0.2)
    labels = []
    for label, a in zip(ax.get_xticklabels(),angles):
        lab = ax.text(a,1.1, label.get_text(),ha=label.get_ha(), va=label.get_va(), size=xsize)
        lab.set_rotation(-np.rad2deg(a))
    ax.set_xticklabels([])
    


In [ ]:
class_dic = {"dist_int_nb_self": 'full',
            'cc': 'cellcycle',
             'int': 'internal',
    
             "dist_nb_self" : 'full nb',
             'nbcc': 'cellcycle nb',
             'self': 'autocorr',
             'nb': 'nb',
             'dist': 'dist-to-rim'}

In [ ]:
dat_mod_stats

In [ ]:
V.COL_CONDNAME

In [ ]:
V.COL_MODELCLASS_F2 = V.COL_MODELCLASS+'_factor2'
V.COL_CONDNAME_F = V.COL_CONDNAME + 'factor'
V.COL_CONDNAME_FR = V.COL_CONDNAME_F + 'R'
dat_mod_stats[V.COL_MODELCLASS_F2] =  pd.Categorical(dat_mod_stats[V.COL_MODELCLASS],
                                                                        categories=
                                                                        ["dist_int_nb_self",
                                                                         'int',
                                                                          'cc',
                                                                         "dist_nb_self",
                                                                         'self',
                                                                         'nb',
                                                                         'dist'
                                                                        
                                                                         
                                                                         ])

cell_ord = dat_mod_stats[V.COL_CONDNAME].unique()
dat_mod_stats[V.COL_CONDNAME_F] = pd.Categorical(dat_mod_stats[V.COL_CONDNAME] ,
                                                    categories=cell_ord
                                                   )
dat_mod_stats[V.COL_CONDNAME_FR] = pd.Categorical(dat_mod_stats[V.COL_CONDNAME] ,
                                                    categories=reversed(cell_ord)
                                                   )

dat_mod_stats[V.COL_MODELCLASS_F2] = dat_mod_stats[V.COL_MODELCLASS_F2].values.rename_categories(class_dic)

In [ ]:
ref_class = 'dist_nb_self'
V.COL_GOODNAME_S = V.COL_GOODNAME + '_sorted'

chan = V.COL_IDCHANNAME
sort_val = dat_mod_stats.loc[dat_mod_stats[V.COL_MODELCLASS] == ref_class,:].groupby(chan)[V.COL_R2].max().reset_index()
dat_mod_stats[V.COL_GOODNAME_S] = pd.Categorical(dat_mod_stats[chan],
                  categories=sort_val.sort_values(V.COL_R2, ascending=False)[chan].tolist(), ordered=True
)

In [ ]:
dat_mod_stats[V.COL_IDCHANNAME] = dat_mod_stats.apply(lambda x: x[V.COL_GOODNAME] + ' - ' + x[V.COL_CHANNELNAME], axis=1)

cdat = dat_mod_stats.pivot_table(columns=V.COL_IDCHANNAME, values='r2_corr', index=[V.COL_CONDNAME_FR,V.COL_MODELCLASS])

rlink =  hclust.linkage(cdat.T, method='ward',metric='euclidean')

marklvl = cdat.columns[hclust.leaves_list(rlink)]

In [ ]:
dat_mod_stats[V.COL_CONDNAME].unique()

In [ ]:
sns.set_context('notebook')
with sns.axes_style("whitegrid",  {'grid.color': '0.1', 'axes.edgecolor': '0.',
                                  'grid.linestyle': 'dotted'}) as s:
    g = sns.FacetGrid(dat_mod_stats.query(f'{V.COL_CONDNAME} in {C.main_conds}')
                      .merge(dat_condmeta), col=V.COL_GOODNAME_S, col_wrap=12,
                      subplot_kws=dict(projection='polar'),
                      sharex=False, sharey=False, despine=False,
                     hue=V.COL_CELLLINE, height=5, legend_out=True)

    # Draw a scatterplot onto each axes in the grid
    g.map(make_spider, V.COL_MODELCLASS_F2, r_var,hue=V.COL_CELLLINE).add_legend()
    g.set_titles(row_template = '{row_name}', col_template = '{col_name}', size=16, weight='bold')
    g.set_xlabels('')
    for ax in g.axes:
        ax.title.set_position([.5, 1.1])
        ax.yaxis.labelpad = 26

In [ ]:
sns.set_context('notebook')
with sns.axes_style("whitegrid",  {'grid.color': '0.1', 'axes.edgecolor': '0.',
                                  'grid.linestyle': 'dotted'}) as s:
    sns.set_palette("deep")
    g = sns.FacetGrid(dat_mod_stats.query(f'{V.COL_CONDNAME} in {C.main_conds}')
                      .merge(dat_condmeta), col=V.COL_GOODNAME_S, col_wrap=7,
                      subplot_kws=dict(projection='polar'),
                      sharex=False, sharey=False, despine=False,
                     hue=V.COL_CELLLINE, height=4)

    # Draw a scatterplot onto each axes in the grid
    g.map(make_spider, V.COL_MODELCLASS_F2, r_var, xsize=17)
    g.set_titles(row_template = '{row_name}', col_template = '{col_name}', size=16, weight='bold')
    g.set_xlabels('')
    for ax in g.axes:
        ax.title.set_position([.5, 1.1])
        ax.yaxis.labelpad = 26
        ax.xaxis.axes.set_xlabel('')
    #plt.tight_layout()
    g.set_xlabels('')
    g.set_ylabels('')
    g.add_legend()

g.savefig(fol_out_paper/'spyder_all.pdf')

In [ ]:
dat_fil = ((dat_mod_stats.groupby(V.COL_GOODNAME_S)[r_var].max() > 0.4)
            .rename('fil')
              .reset_index())
dat_fil = dat_fil.query('fil == True')
V.COL_GOODNAME_S_F = V.COL_GOODNAME_S+'fil'

def adapt_missing_categories(catcol):
    vals = np.unique(catcol.values)
    cat = [c for c in pd.Categorical(catcol).categories if c in vals]
    return pd.Categorical(catcol,categories=cat)

dat_fil[V.COL_GOODNAME_S_F] = adapt_missing_categories(dat_fil[V.COL_GOODNAME_S])

Sort the markers by average variablity

In [ ]:
V.COL_RELVAR = 'rel_r2'
ref_class ='dist_nb_self'
V.COL_GOODNAME_S_VAR = V.COL_GOODNAME_S_F+'var'
V.COL_VAR = 'var'
dat_mod_stats[V.COL_RELVAR] = dat_mod_stats.groupby(V.COL_GOODNAME)[r_var].transform(lambda x: x/x.max())



In [ ]:
dat_varname = (dat_mod_stats
 .merge(dat_fil)
 .groupby([V.COL_GOODNAME, V.COL_MODELCLASS_F2])[V.COL_RELVAR].apply(lambda x: np.std(x))
 .groupby(V.COL_GOODNAME).max()
 .rename(V.COL_VAR)
 .sort_values(ascending=False)
  .reset_index()
)
dat_varname[V.COL_GOODNAME_S_VAR] = pd.Categorical(dat_varname[V.COL_GOODNAME],categories=dat_varname[V.COL_GOODNAME])
trow_var = 'row'
dat_varname[trow_var] = [i % 3 for i in range(dat_varname.shape[0])]

In [ ]:
sns.set_context('notebook')
fns_out = [os.path.join(fol_out, f'spiderplots_row{f}.png') for f in range(3)]
for r in range(3):
    with sns.axes_style("whitegrid",  {'grid.color': '0.1', 'axes.edgecolor': '0.',
                                      'grid.linestyle': 'dotted'}) as s:
        sns.set_palette("deep")
        tdat = (dat_mod_stats.merge(dat_fil).merge(dat_varname).query(f'{trow_var} == {r}').query(f'{V.COL_CONDNAME} in {C.main_conds}')
                      .merge(dat_condmeta))
        tdat['col'] = adapt_missing_categories(tdat[V.COL_GOODNAME_S])
        g = sns.FacetGrid(tdat, 
                          col='col', col_wrap=11,
                          subplot_kws=dict(projection='polar'),
                          sharex=False, sharey=False, despine=False,
                         hue=V.COL_CELLLINE, height=3.5)

        # Draw a scatterplot onto each axes in the grid
        g.map(make_spider, V.COL_MODELCLASS_F2, r_var, xsize=17)
        g.set_titles(row_template = '{row_name}', col_template = '{col_name}', size=16, weight='bold')
        g.set_xlabels('')
        for ax in g.axes:
            ax.title.set_position([.5, 1.1])
            ax.yaxis.labelpad = 26
            ax.xaxis.axes.set_xlabel('')
        g.fig.subplots_adjust(top=.8)
        g.fig.savefig(fns_out[r])
    #plt.tight_layout()
    


In [ ]:
def cat_cond(col):
    return pd.Categorical(col, categories=["DLD1", "T47D", "293T", 'HT29'])
(gg.ggplot(dat_mod_stats.loc[dat_mod_stats[V.COL_MODELCLASS] == 'dist_int_nb_self',:],
          gg.aes(x=r_var, fill=V.COL_CONDNAME_F))+
         gg.facet_grid(V.COL_CONDNAME_F+'~.')+
         gg.geom_histogram(color='black', breaks= [x/10+0.1 for x in range(10)])+
        gg.xlim((0,1))+
        gg.xlab('R2')+
        gg.theme_seaborn(style='whitegrid',context='talk')+
        gg.scale_color_cmap())

In [ ]:
def doplot(x, y, color, label):
    ax = plt.gca()
    xval = x.values.codes
    o = np.argsort(xval)
    y = y.values
    ax.plot(xval[o], y[o], color=color, linewidth=2, linestyle='solid')
    ax.scatter(xval[o], y[o], color=color, linewidth=2, linestyle='solid')
    ax.fill_between(xval[o], 0, y[o], color=color, alpha=0.2)
    

    ax.set_xticks(np.arange(len(xval)))
    ax.set_xticklabels(x.values.categories,rotation=90)
    plt.yticks([0, 0.25,0.5,0.75,1], size=10)
    
    plt.ylim(0,1)

In [ ]:
example_mark = ['Er166', 'Nd150','Yb171', 'Er168']
dic = {r[V.COL_CHANNELNAME]: r[V.COL_GOODNAME] for i,r in dat_mod_stats.iterrows() }
example_goodname = [dic[g] for g in example_mark]
V.COL_GOODNAME_SLECT = V.COL_GOODNAME +'select'

dat_mod_stats[V.COL_GOODNAME_SLECT] = pd.Categorical(dat_mod_stats[V.COL_GOODNAME],
                                                  categories=example_goodname)

In [ ]:
sns.set_context('talk')
with sns.axes_style("whitegrid", {'grid.color': '0.1', 'axes.edgecolor': '0.',
                                  'grid.linestyle': 'dotted'}) as s:
    g = sns.FacetGrid(dat_mod_stats.loc[(dat_mod_stats[V.COL_CHANNELNAME].isin(example_mark)) &
                             (dat_mod_stats[V.COL_MODELCLASS_F2].isna() == False)].query(f'{V.COL_CONDNAME} in {C.main_conds}')
                      .merge(dat_condmeta),
                      col=V.COL_GOODNAME_SLECT,
                      sharex=False, sharey=False, despine=False,
                     hue=V.COL_CELLLINE)

    # Draw a scatterplot onto each axes in the grid
    g.map(doplot, V.COL_MODELCLASS_F2, V.COL_R2)
    g.set_titles(row_template = '{row_name}', col_template = '{col_name}',
             size=16, weight='bold')
    g.set_xlabels('')
    #g.map(plt.fill, V.COL_MODELCLASS_F2, V.COL_R2,color=color, alpha=0.2)

In [ ]:

with sns.axes_style("whitegrid",  {'grid.color': '0.1', 'axes.edgecolor': '0.',
                                  'grid.linestyle': 'dotted'}) as s:
    sns.set_palette("deep")
    g = sns.FacetGrid(dat_mod_stats.loc[(dat_mod_stats[V.COL_CHANNELNAME].isin(example_mark)) &
                             (dat_mod_stats[V.COL_MODELCLASS_F2].isna() == False)].query(f'{V.COL_CONDNAME} in {C.main_conds}')
                      .merge(dat_condmeta),
                      col=V.COL_GOODNAME_SLECT, col_wrap=6,
                      subplot_kws=dict(projection='polar'),
                      sharex=False, sharey=False, despine=False,
                     hue=V.COL_CELLLINE, height=4)

    # Draw a scatterplot onto each axes in the grid
    g.map(make_spider, V.COL_MODELCLASS_F2, V.COL_R2)
    g.set_titles(row_template = '{row_name}', col_template = '{col_name}', size=16, weight='bold')
    for ax in g.axes:
        ax.title.set_position([.5, 1.1])
        ax.yaxis.labelpad = 25
    g.set_xlabels('')
    g.set_ylabels('')
g.savefig(fol_out_paper/'spyderexamples.pdf')

In [ ]:

with sns.axes_style("whitegrid",  {'grid.color': '0.1', 'axes.edgecolor': '0.',
                                  'grid.linestyle': 'dotted'}) as s:
    sns.set_palette("deep")
    tmpdat = (dat_mod_stats.loc[(dat_mod_stats[V.COL_CHANNELNAME].isin(example_mark)) &
                             (dat_mod_stats[V.COL_MODELCLASS_F2].isna() == False)]
              .query(f'{V.COL_CONDNAME} in {C.main_conds}')
                      .merge(dat_condmeta))
    for cl, grp in tmpdat.groupby(V.COL_CELLLINE):
        g = sns.FacetGrid(grp,
                          col=V.COL_GOODNAME_SLECT, col_wrap=6,
                          subplot_kws=dict(projection='polar'),
                          sharex=False, sharey=False, despine=False,
                         hue=V.COL_CELLLINE, height=4)

        # Draw a scatterplot onto each axes in the grid
        g.map(make_spider, V.COL_MODELCLASS_F2, V.COL_R2)
        g.set_titles(row_template = '{row_name}', col_template = '{col_name}', size=16, weight='bold')
        for ax in g.axes:
            ax.title.set_position([.5, 1.1])
            ax.yaxis.labelpad = 25
        g.set_xlabels('')
        g.fig.suptitle(cl, size=16)
        g.fig.subplots_adjust(top=.9)

In [ ]:

with sns.axes_style("whitegrid",  {'grid.color': '0.1', 'axes.edgecolor': '0.',
                                  'grid.linestyle': 'dotted'}) as s:
    g = sns.FacetGrid(dat_mod_stats.loc[(dat_mod_stats[V.COL_CHANNELNAME].isin(example_mark)) &
                             (dat_mod_stats[V.COL_MODELCLASS_F2].isna() == False)& 
                                       (dat_mod_stats[V.COL_CONDNAME] == 'HT29')],
                      col=V.COL_GOODNAME_SLECT, col_wrap=6,
                      subplot_kws=dict(projection='polar'),
                      sharex=False, sharey=False, despine=False,
                     hue=V.COL_CONDNAME_FR)

    # Draw a scatterplot onto each axes in the grid
    g.map(make_spider, V.COL_MODELCLASS_F2, V.COL_R2)
    g.set_titles(row_template = '{row_name}', col_template = '{col_name}', size=16, weight='bold')
    for ax in g.axes:
        ax.title.set_position([.5, 1.1])
        ax.yaxis.labelpad = 25
    g.set_xlabels('')

In [ ]:
dat_mod_stats[V.COL_MODELCLASS_F2]

In [ ]:

def get_code(c):
    return c.values.codes
dat_mod_stats['ymin'] = 0
(gg.ggplot(dat_mod_stats.loc[(dat_mod_stats[V.COL_CHANNELNAME].isin(example_mark)) &
                             (dat_mod_stats[V.COL_MODELCLASS_F2].isna() == False)],
          gg.aes(x='get_code('+V.COL_MODELCLASS_F2+')',y=V.COL_R2, 
                 fill=V.COL_CONDNAME_F,
                color=V.COL_CONDNAME_F))+
         gg.facet_grid(V.COL_GOODNAME_S+'~.')+
         #gg.geom_ribbon(gg.aes(ymin='ymin',alpha=0.2,))+
        gg.geom_line()+

        gg.theme_seaborn(style='whitegrid',context='poster'))

In [ ]:
fil = (dat_mod_stats[V.COL_MODELCLASS_F2].isna() == False)

dat_hm = dat_mod_stats.loc[fil].pivot_table(index=[V.COL_CHANNELNAME, V.COL_CONDNAME, V.COL_GOODNAME],values=V.COL_R2, columns=V.COL_MODELCLASS_F2)
#dat_hm = dat_pca.query('dist_int_nb_self > 0.3')

Load per-condition data

In [ ]:
dat_mod_repstats = pd.read_csv(fn_modstats_perrep).rename(columns={V.COL_CONDNAME: V.COL_CONDID})

In [ ]:
dat_condtab = bro.doquery(bro.session.query(db.conditions)).rename(columns={'well_name': 'well'})

In [ ]:
V.COL_MODELCLASS_F

In [ ]:
dat_layout

In [ ]:
V.COL_R2

In [ ]:
rcol = V.COL_R2
chans=["Yb171", "Er168"]
(dat_mod_repstats.merge(dat_condtab).merge(dat_layout)
 .query(f'{V.COL_MODELCLASS} in {cluster_models+["dist_int_nb_self"]}')
 .query(f'{V.COL_CHANNELNAME} in {chans} ')>> 
 gg.ggplot(gg.aes(x=V.COL_MODELCLASS, y=rcol , color=f'pd.Categorical({V.COL_TP})'))+
 gg.facet_grid(f'{V.COL_CELLLINE}~{V.COL_CONC}+{V.COL_CHANNELNAME}')+
 gg.geom_boxplot(outlier_alpha=0)+
 gg.geom_point()+
 gg.geom_point((dat_mod_stats.merge(dat_condmeta)
              .query(f'{V.COL_MODELCLASS} in {cluster_models+["dist_int_nb_self"]}')
 .query(f'{V.COL_CHANNELNAME} in {chans}')), color='black', size=2)+
 gg.theme(figure_size=(14,14))
 
)

In [ ]:
rcol = 'r2_adj_corr'
chans=["Yb171", "Er168"]
(dat_mod_repstats.merge(dat_condtab).merge(dat_layout)
 .query(f'{V.COL_MODELCLASS} in {cluster_models+["dist_int_nb_self"]}')
 .query(f'{V.COL_CHANNELNAME} in {chans} ')>> 
 gg.ggplot(gg.aes(x=V.COL_MODELCLASS, y=rcol , color=f'pd.Categorical({V.COL_TP})'))+
 gg.facet_grid(f'{V.COL_CELLLINE}~{V.COL_CONC}+{V.COL_CHANNELNAME}')+
 gg.geom_boxplot(outlier_alpha=0)+
 gg.geom_point()+
 gg.geom_point((dat_mod_stats.merge(dat_condmeta)
              .query(f'{V.COL_MODELCLASS} in {cluster_models+["dist_int_nb_self"]}')
 .query(f'{V.COL_CHANNELNAME} in {chans}')), color='black', size=2)+
 gg.coord_cartesian(ylim=(0,1))+
 gg.theme(figure_size=(14,14))
 
)

In [ ]:
(dat_mod_repstats.merge(dat_condtab).merge(dat_layout).merge(dat_mod_stats[[V.COL_MODELCLASS, V.COL_MODELCLASS_F]].drop_duplicates().dropna())
 .query(f'{V.COL_MODELCLASS} in {cluster_models+["dist_int_nb_self"]}')
 .query(f'{V.COL_CHANNELNAME} in {chans} '))

In [ ]:
rcol = 'r2_adj_corr'
chans=["Yb171", "Er168", "Er166"]
(dat_mod_repstats.merge(dat_condtab).merge(dat_layout).merge(dat_mod_stats[[V.COL_MODELCLASS, V.COL_MODELCLASS_F]].drop_duplicates().dropna(subset=[V.COL_MODELCLASS_F]))
 .query(f'{V.COL_MODELCLASS} in {cluster_models+["dist_int_nb_self"]}')
 .query(f'{V.COL_CHANNELNAME} in {chans} ')>> 
 gg.ggplot(gg.aes(x=V.COL_CONC, y=rcol , color=f'pd.Categorical({V.COL_TP})'))+
 gg.facet_grid(f'{V.COL_CELLLINE}~{V.COL_CHANNELNAME}+{V.COL_MODELCLASS}')+
 gg.geom_boxplot(outlier_alpha=0)+
 gg.geom_point()+
 gg.geom_point((dat_mod_stats.merge(dat_condmeta).dropna(subset=[V.COL_MODELCLASS_F])
              .query(f'{V.COL_MODELCLASS} in {cluster_models+["dist_int_nb_self"]}')
 .query(f'{V.COL_CHANNELNAME} in {chans}')), color='black', size=2)+
 gg.coord_cartesian(ylim=(0,1))+
 gg.theme(figure_size=(14,14))
 
)

In [ ]:
rcol = 'r2_adj_corr'
chans=["Yb171", "Er168", "Er166"]
(dat_mod_repstats.merge(dat_condtab).merge(dat_layout).merge(dat_mod_stats[[V.COL_MODELCLASS, V.COL_MODELCLASS_F]].drop_duplicates().dropna(subset=[V.COL_MODELCLASS_F]))
 .query(f'{V.COL_MODELCLASS} in {cluster_models+["dist_int_nb_self"]}')
 .query(f'{V.COL_CHANNELNAME} in {chans} ')>> 
 gg.ggplot(gg.aes(x=V.COL_CONC, y=rcol , color=f'pd.Categorical({V.COL_TP})'))+
 gg.facet_grid(f'{V.COL_MODELCLASS_F}~{V.COL_CELLLINE}+{V.COL_CHANNELNAME}')+
 gg.geom_boxplot(outlier_alpha=0)+
 gg.geom_point()+
 gg.geom_point((dat_mod_stats.merge(dat_condmeta).dropna(subset=[V.COL_MODELCLASS_F])
              .query(f'{V.COL_MODELCLASS} in {cluster_models+["dist_int_nb_self"]}')
 .query(f'{V.COL_CHANNELNAME} in {chans}')), color='black', size=2)+
 gg.coord_cartesian(ylim=(0,1))+
 gg.theme(figure_size=(14,14))
 
)

In [ ]:
dat_condtab

In [ ]:
dat_mod_repstats.merge(dat_condtab).merge(dat_layout)

In [ ]:
(dat_mod_repstats.merge(dat_condtab).merge(dat_layout).merge(dat_mod_stats[[V.COL_MODELCLASS, V.COL_MODELCLASS_F, V.COL_CHANNELNAME, V.COL_GOODNAME]].drop_duplicates().dropna(subset=[V.COL_MODELCLASS_F]))
 .query(f'{V.COL_MODELCLASS} in {cluster_models+["dist_int_nb_self"]}')
 .query(f'{V.COL_CHANNELNAME} in {chans} ')
.query('(modelclass == "dist") & (cellline == "293T") & (concentration==1)'))

In [ ]:

def do_variab_rep_plot(chan='Yb171', rcol='r2_adj_corr',figsize=(14,7)):
    chans = [chan]
    p = (dat_mod_repstats
              .query(f'{V.COL_MODELCLASS} in {cluster_models+["dist_int_nb_self"]}').rename(columns={'No. Observations': 'ncells'})
         .query(f'{V.COL_CHANNELNAME} in {chans} ')
         
         .merge(dat_condtab)
         .merge(dat_layout)
         .merge(dat_mod_stats[[V.COL_MODELCLASS, V.COL_MODELCLASS_F, V.COL_CHANNELNAME, V.COL_GOODNAME]].drop_duplicates().dropna(subset=[V.COL_MODELCLASS_F])) >> 
     gg.ggplot(gg.aes(x=f'pd.Categorical({V.COL_CONC})', y=rcol , fill=f'pd.Categorical({V.COL_TP})'))+
     gg.facet_grid(f'{V.COL_MODELCLASS_F}~{V.COL_GOODNAME}+{V.COL_CELLLINE}+{V.COL_TP}')+
     gg.geom_hline(yintercept=1)+
     gg.geom_hline(yintercept=0)+
     gg.geom_boxplot(outlier_alpha=0, color='gray', fill='white')+
     gg.geom_jitter(gg.aes(size='(ncells)', shape='ncells <100'), alpha=0.8, height=0, width=0.2)+
     gg.scale_size_area(max_size=4, breaks=(100,150,200,400))+
     gg.geom_point((dat_mod_stats.merge(dat_condmeta).dropna(subset=[V.COL_MODELCLASS_F])
                  .query(f'{V.COL_MODELCLASS} in {cluster_models+["dist_int_nb_self"]}')
     .query(f'{V.COL_CHANNELNAME} in {chans}')), color='black', size=2, fill='black')+
     #gg.coord_cartesian(ylim=(0,1))+
     gg.theme(figure_size=(14,7))+
     gg.ylab('Variability explained [Adj. R2]')+
     gg.xlab('Size')

    )
    return p

do_variab_rep_plot()
    


In [ ]:
do_variab_rep_plot(rcol='r2_corr')

In [ ]:
tdat = (dat_mod_repstats
         .merge(dat_condtab)
         .merge(dat_layout)
        .rename(columns={'No. Observations': 'ncells'})
        .query('ncells > 100')
         .groupby(by=[V.COL_MODELCLASS, V.COL_CHANNELNAME, V.COL_CELLLINE, V.COL_TP, V.COL_CONC],observed=True)['r2_adj_corr']
         .agg(**{'std': lambda x: np.std(x, ddof=1),
                'mn': np.mean,
                'md': np.median,
                'n': len}).reset_index()
       .query('n > 3')
         .merge(dat_mod_stats[[V.COL_MODELCLASS, V.COL_MODELCLASS_F, V.COL_CHANNELNAME, V.COL_GOODNAME]].drop_duplicates())
      )





In [ ]:
(gg.ggplot(tdat, gg.aes(x=V.COL_MODELCLASS, y='std', color=V.COL_CELLLINE))+
    gg.geom_boxplot(color='gray', fill='gray')+
    # gg.geom_violin()
    gg.geom_jitter(height=0, width=0.3)
    )

In [ ]:
(gg.ggplot(tdat.query(f'{V.COL_MODELCLASS_F} in {cluster_models+["dist_int_nb_self"]}'), gg.aes(x=V.COL_CHANNELNAME, y='std', color=V.COL_CELLLINE))+
     gg.facet_grid(f'{V.COL_MODELCLASS}~.')+
    gg.geom_boxplot(color='gray', fill='gray')+
    gg.geom_point()
    )

In [ ]:
(gg.ggplot(tdat.query(f'{V.COL_MODELCLASS_F} in {cluster_models+["dist_int_nb_self"]}'), gg.aes(x='mn', y='std', color=V.COL_CELLLINE))+
     gg.facet_grid(f'{V.COL_MODELCLASS_F}~.')+
    gg.geom_point()
    )

In [ ]:
(gg.ggplot(tdat.query(f'{V.COL_MODELCLASS} in {cluster_models+["dist_int_nb_self"]+["dist_nb_self_cc"]}'), gg.aes(x='mn', y='std/np.abs(mn)'))+
     gg.facet_grid(f'{V.COL_MODELCLASS}~.')+
    gg.geom_point(alpha=0.1)+
 gg.coord_cartesian(ylim=(0,1))
    )

In [ ]:
tsum = tdat.groupby(by=[V.COL_MODELCLASS]).mean()
print(tsum)

In [ ]:
tsum = tdat.query('mn>0.05').groupby(by=[V.COL_MODELCLASS]).mean()
print(tsum)

In [ ]:
tsum.eval('std/mn')

In [ ]:
tdat.query('std>0.2').groupby([V.COL_GOODNAME],observed=True)['std'].describe()

In [ ]:
(dat_mod_repstats.merge(dat_condtab).merge(dat_layout).merge(dat_mod_stats[[V.COL_MODELCLASS, V.COL_MODELCLASS_F]].drop_duplicates().dropna(subset=[V.COL_MODELCLASS_F]))
 .query(f'{V.COL_MODELCLASS} in {cluster_models+["dist_int_nb_self"]}')
 .query(f'{V.COL_CHANNELNAME} in {chans} '))

Check cell sizes:

In [ ]:

COL_CELLLINE = 'cellline'
COL_CONC = 'conc'
COL_ISTELOX = 'istelox'
COL_TP = 'tp'
def split_names(x):
    c = re.compile('(?P<{}>.*)_c(?P<{}>.*)_tp(?P<{}>.*)'.format(COL_CELLLINE, COL_CONC, COL_TP))
    
    m = c.match(x)
    g = m.groups() 
    return pd.Series({l: g[i-1] for l, i in c.groupindex.items()},name=x.index)



In [ ]:
fn_size = sm.input.fn_bf_quantification
# variables bfanalysis
COL_BF_AREA = 'AreaShape_Area'
COL_BF_WELL = 'Metadata_well'
COL_BF_PLATE = 'Metadata_plate'
COL_RADIUS = 'radius'

dat_conditions=bro.doquery(bro.session.query(db.conditions))
dat_conditions = dat_conditions.join(dat_conditions[V.COL_CONDNAME].apply(split_names))
dat_conditions[COL_CONC] = dat_conditions[COL_CONC].astype(np.float)
dat_size = pd.read_csv(fn_size)
BF_RESOLTUION = 5*0.65
def get_radius(area, resolution=BF_RESOLTUION):
    r = np.sqrt(area/np.pi)
    return r*resolution

dat = dat_size.merge(dat_conditions, left_on=[COL_BF_PLATE,COL_BF_WELL], right_on=[db.conditions.plate_id.key, db.conditions.well_name.key] )

In [ ]:
dat[COL_RADIUS] = dat[COL_BF_AREA].map(get_radius)

In [ ]:
p=(gg.ggplot(dat, gg.aes(x=COL_CELLLINE, y=COL_RADIUS, fill=COL_TP))+
     gg.facet_wrap(f'{COL_CONC}')+
     gg.geom_boxplot(outlier_size=0)+
     gg.geom_point(position=gg.position_dodge(width=0.5))+
    gg.expand_limits(y=0)+
    gg.xlab('Cell line')+
    gg.ylab('Radius [um]')+
    gg.ggtitle('Spheroid Size before harvesting\n(assessed by brightfield imaging)'))

p

In [ ]:
(gg.ggplot(dat, gg.aes(x=COL_CONC, y=COL_RADIUS, color=COL_TP))+
     gg.facet_wrap(f'{COL_CELLLINE}')+
     #gg.geom_boxplot(outlier_size=0, position='dodge')+
     gg.geom_line()+
     gg.geom_jitter(height=0, width=0.05)+
    gg.expand_limits(y=0)+
    gg.xlab('Cell line')+
    gg.ylab('Radius [um]')+
    gg.ggtitle('Spheroid Size before harvesting\n(assessed by brightfield imaging)'))

In [ ]:
p=(gg.ggplot(dat, gg.aes(x=COL_CELLLINE, y=COL_RADIUS, fill=COL_TP))+
     gg.facet_wrap(f'{COL_CONC}')+
     gg.geom_boxplot(outlier_size=0)+
     gg.geom_point(position=gg.position_dodge(width=0.5))+
    gg.expand_limits(y=0)+
    gg.xlab('Cell line')+
    gg.ylab('Radius [um]')+
    gg.ggtitle('Spheroid Size before harvesting\n(assessed by brightfield imaging)'))
gg.ggsave(p, fol_out_paper / 'spheroid_sizes.pdf')

p

Equation
$y_{pi} = \beta_{pi} +BS(d^{distrim})+ \sum_{m\neq p}\beta^{nb}_{m}x^{nb}_m +\beta_p^{nb}x^{nb}_p + \sum_{m\neq p}\beta^{int}_{m}x^{int}_m + \epsilon_{pi}$